- 所有文件都会先经过ocr识别处理
- ocr后的结果应该是一个大的json文件，该文件应：
    - 嵌套式体现文档结构，表明哪些是标题及其标题等级
    - 文档内容应区分表格内容（有开头结尾标记）和其他
    - 内容跨页需要合并
- 由于ocr接口正在开发中，本方案依据理想化的ocr结果编写

In [2]:
from paddlenlp import Taskflow
import pandas as pd
import json
from IPython.display import display
import re
import os
from translate import Translator

E:\miniconda3\envs\ZheShang\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
E:\miniconda3\envs\ZheShang\lib\site-packages\_distutils_hack\__init__.py:32: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(


In [3]:
##忽视future warnings
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

# Params & Helping Functions

## extract_info_from_taskflow

In [4]:
ie = Taskflow('information_extraction', task_path = './uie-base') 
## if from pretrain: task_path = 'uie-base path' 
## 现在模型在 "C:/Users/0049004320/.paddlenlp/taskflow/information_extraction/uie-base"

def extract_info_from_taskflow(case, schema, task=ie):
    '''
    task: taskflow
    case: string, info for analysing
    schema: key-words needs to be extracted
    '''
    task.set_schema(schema)
    info = task(case)  ## paddleNLP taskflow 模型抽取信息
    result = {}  ## 存放抽取结果
    for k in schema:
        v = info[0].get(k)
        if v:
            v = v[0].get("text")
        result[k] = v
    return result

[2024-09-05 08:37:09,371] [ WARNING] - The schema has not been set yet, please set a schema via set_schema(). More details about the setting of schema please refer to https://github.com/PaddlePaddle/PaddleNLP/blob/develop/applications/information_extraction/taskflow_text.md
[2024-09-05 08:37:11,404] [    INFO] - We are using <class 'paddlenlp.transformers.ernie.tokenizer.ErnieTokenizer'> to load './uie-base'.


## key translation

In [85]:
info_extraction = {
'account_information':[{'issuer': '工商银行信用卡中心', 'account_identification': 'AAA003', 'date_of_issue': '2009.05.22', 'account_credit_limit': '16,000', 'shared_credit_limit': '16,000', 'currency': '人民币', 'business_type': '贷记卡', 'guarantee': '信用/免担保'}],
'as_of_april_12_2015':[{'account_status': '正常', 'balance': '255,500', 'used_credit_limit': '135,500', 'large_amount_of_unissued_special_installment_balance': '110,000', 'number_of_remaining_installments': '11', 'average_usage_limit_last_6_months': '90,000', 'maximum_usage_limit': '150,000'}, {'statement_closing_date': '2015.04.12', 'repayment_due_this_month': '24,500', 'actual_repayment_this_month': '0', 'last_repayment_date': '2015.03.10', 'current_overdue_periods': '1', 'current_overdue_total': '24,550'}],
'large_amount_of_special_installment_information':[{'large_amount_of_special_installment_quota': '120,000', 'installment_limit_effective_date': '2015.09.12', 'installment_limit_expiration_date': '2016.09.11', 'spent_installment_amount': '120,000'}],
'latest_repayment_history_after_april_12_2015':[{'balance': '230,950', 'repayment_date': '2015.04.25', 'repayment_amount': '24,550', 'current_repayment_status': 'N'}],
'repayment_history_from_july_2010_to_april_2015':[
{'2015': [{'1':'N','2':'N','3':'N','4':'N'},{'1':'0','2':'0','3':'0','4':'0'}]},
{'2014':[{'1':'N','2':'N','3':'N','4':'N','5':'N','6':'N','7':'N','8':'N','9':'N','10':'N','11':'N','12':'N'},{'1':'0','2':'0','3':'0','4':'0','5':'0','6':'0','7':'0','8':'0','9':'0','10':'0','11':'0','12':'0'}]},
{'2013': [{'1':'N','2':'N','3':'N','4':'N','5':'N','6':'N','7':'N','8':'N','9':'N','10':'N','11':'N','12':'N'},{'1':'0','2':'0','3':'0','4':'0','5':'0','6':'0','7':'0','8':'0','9':'0','10':'0','11':'0','12':'0'}]},
{'2012': [{'1':'N','2':'N','3':'N','4':'N','5':'N','6':'N','7':'N','8':'N','9':'N','10':'N','11':'N','12':'N'},{'1':'0','2':'0','3':'0','4':'0','5':'0','6':'0','7':'0','8':'0','9':'0','10':'0','11':'0','12':'0'}]},
{'2011': [{'1':'#','2':'#','3':'#','4':'#','5':'#','6':'#','7':'#','8':'#','9':'#','10':'#','11':'#','12':'#'},{'1':'--','2':'--','3':'--','4':'--','5':'--','6':'--','7':'--','8':'--','9':'--','10':'--','11':'--','12':'--'}]},
{'2010': [{'7':'N','8':'N','9':'N','10':'N','11':'N','12':'N'},{'7':'0','8':'0','9':'0','10':'0','11':'0','12':'0'}]}],
'special_event_instructions':[{'special_event_instructions':'该账户 2015 年 05 月不出单。','special_transaction_type':'个性化分期','date_of_occurrence':'2012.07','change_months':'5','incurred_amount':'10,000','detailed_records':'--'}],
'institution_description':[{'institution_description':'该卡逾期情况极为严重。','date_added':'2012.12.15'}],
'i_certify_that':[{'i_certify_that':'本人因出国无法及时还款，导致当前逾期，非恶意违约。','date_added':'2013.07.02'}],
'objection_annotation':[{'objection_annotation':'该笔业务正处于异议处理中。','date_added':'2013.07.22'}]}

info_extraction

{'account_information': [{'issuer': '工商银行信用卡中心',
   'account_identification': 'AAA003',
   'date_of_issue': '2009.05.22',
   'account_credit_limit': '16,000',
   'shared_credit_limit': '16,000',
   'currency': '人民币',
   'business_type': '贷记卡',
   'guarantee': '信用/免担保'}],
 'as_of_april_12_2015': [{'account_status': '正常',
   'balance': '255,500',
   'used_credit_limit': '135,500',
   'large_amount_of_unissued_special_installment_balance': '110,000',
   'number_of_remaining_installments': '11',
   'average_usage_limit_last_6_months': '90,000',
   'maximum_usage_limit': '150,000'},
  {'statement_closing_date': '2015.04.12',
   'repayment_due_this_month': '24,500',
   'actual_repayment_this_month': '0',
   'last_repayment_date': '2015.03.10',
   'current_overdue_periods': '1',
   'current_overdue_total': '24,550'}],
 'large_amount_of_special_installment_information': [{'large_amount_of_special_installment_quota': '120,000',
   'installment_limit_effective_date': '2015.09.12',
   'installmen

In [10]:
a = '本人版'

key_en=translator.translate(a)
 ## 原有下划线为标题分级
key_en = re.sub(r'\_', '\_\_', key_en)  
## 转小写
## 删除非英文字母非空格的字符
key_en = re.sub(r'[^a-zA-Z0-9\_\s]', '', key_en.lower())  
## 下划线连接
key_en = '_'.join(key_en.split())
key_en

'my_version'

In [77]:
trans_dict_path = "./data/key2en.json"
translator = Translator(from_lang="ZH",to_lang="EN")

def key_2_English(key, translator=translator, 
                  trans_dict_path=trans_dict_path, 
                  force_translate=False, update_trans_dict_file=True):
    '''
    用translate包将key转换成英文
    并维护一个中转英的dict
    可以人为修改对应英文

    key: 一个要翻译的字符串
    translator: Translator(from_lang="ZH",to_lang="EN")
    trans_dict_path: 存储中英字符对应翻译

    force_translate: 强制重新翻译，也会更新表格
    update_trans_dict_file：是否更新存储的dict
    '''
    ## 检查是否含有中文
    contains_ZH = False
    for char in key:
        if '\u4e00' <= char <= '\u9fff':
            contains_ZH = True
            break
    if not contains_ZH:
        return key
        
    need_updating = False  ## flag for 是否需要更新字典

    ## 载入已有dict, 或创立新dict
    if not os.path.exists(trans_dict_path):
        need_updating = True
        trans_dict = {}
    else:
        with open(trans_dict_path, 'r') as file:
            trans_dict = json.load(file)

    key_en = trans_dict.get(key)
    if force_translate:
        key_en = None
    if not key_en:
        need_updating = True
        key_en = translator.translate(key)

        matches = re.findall(">(.*?)<", key_en)  ## 有时会有<bold>..<>修饰
        if matches:
            key_en = matches[0].strip(":")

        ## 原有下划线为标题分级
        key_en = re.sub(r'\_', '\_\_', key_en)  
        ## 转小写
        ## 删除非英文字母非空格的字符
        key_en = re.sub(r'[^a-zA-Z0-9\_\s]', '', key_en.lower())  
        ## 下划线连接
        key_en = '_'.join(key_en.split())
        
        ## 更新dict
        trans_dict[key] = key_en
        if update_trans_dict_file:
            with open(trans_dict_path, 'w') as file:
                json.dump(trans_dict, file)

    return key_en

def update_trans_dict(key, key_en, trans_dict_path=trans_dict_path):
    '''
    手动修改英文翻译
    '''
    need_updating = False

    ## 载入已有dict, 或创立新dict
    if not os.path.exists(trans_dict_path):
        need_updating = True
        trans_dict = {}
    else:
        with open(trans_dict_path, 'r') as file:
            trans_dict = json.load(file)

    ## 更新dict
    trans_dict[key] = key_en
    with open(trans_dict_path, 'w') as file:
        json.dump(trans_dict, file)

def key_list_2_English(keys):
    return [key_2_English(k) for k in keys]

## split cases

In [88]:
def split_cases_by_idx_from_paragraph(data, split_pattern=r'\s+\d+\.\s*'): 
    '''
    用于分段：
        1. 。。。
        2. 。。。
    '''
    data = '\n' + data.strip().strip("\n")

    ## 分段标准: 一个或多个空白 + 一个或多个数字 + . + 空格
    data_splited = re.split(split_pattern, data)

    cases = []
    for case in data_splited:
        if case == '':
            continue
        ## 去所有\n和空格
        case = case.replace("\n","").replace(" ", "")
        cases.append(case)
    return cases

## text result fixing

In [194]:
def text_result_fixing(case, result):
    case = case.replace(" ", "")
    
    if "截至" not in case:
        if "余额截至日期" in result:
            result["余额截至日期"] = None
        if "截至日期" in result:
            result["截至日期"] = None
    
    if "余额" not in case:
        if "余额数目" in result:
            result["余额数目"] = None
        if "总余额" in result:
            result["总余额"] = None
        
    if "到期" not in case:
        if "贷款到期日期" in result:
            result["贷款到期日期"] = None
        if "业务到期日期" in result:
            result["业务到期日期"] = None

    if "已结清" not in case:
        if "结清日期" in result:
            result["结清日期"] = None

    if "最近一次还款日期" not in case:
        if "最近还款日期" in result:
            result["最近还款日期"] = None

    if '当前透支超过60天' in case:
        if "当前透支是否超过60天" in result:
            result["当前透支是否超过60天"] = '是'
    
    if '已使用额度' in result:
        if '已使用额度' not in case:
            result["已使用额度"] = None


    """
    以下key,taskflow 其实不用处理
    """
    if "是否激活" in result:
        if '尚未激活' in case:
            result["是否激活"] = '否'
        
    if "是否为呆账" in result:
        if "已变成呆账" in case:
            result["是否为呆账"] = "是"
    
    if "是否变成司法追偿" in result:
        if "已变成司法追偿" in case:
            result["是否变成司法追偿"] = '是'
 
    if "当前有无逾期" in result:
        dqywyq = re.search(r'当前(.*?)逾期', case)
        if dqywyq:
            result["当前有无逾期"] = dqywyq.group(1)

    if "是否为其他机构转入" in result:
        if "其他机构转入" in case:
            result["是否为其他机构转入"] = "是"
        
    if "贷款币种" in result:
        dkbz = re.search(r"\（(.*?)\）.*?贷款", case)
        if dkbz:
            result["贷款币种"] = dkbz.group(1)

    if "贷款种类" in result:  ## 获取所有的贷款种类会更好
        dkzl = re.search(r'\）(.*?)贷款', case)
        if dkzl:
            result["贷款种类"]=dkzl.group(1) + "贷款"

    if "业务种类" in result:
        ywzl = case.split("，")
        if ywzl:
            ywzl = ywzl[0].split("）")
            if ywzl:
                ywzl = ywzl[-1]
                result["业务种类"] = ywzl

    if "业务货币种类" in result:
        ywhbzl = re.search(r'办理的(.*?)，', case)
        if ywhbzl:
            ywhbzl = re.search(r'\（(.*?)\）', ywhbzl.group(1))
            if ywhbzl:
                result["业务货币种类"] = ywhbzl.group(1)

    if "业务办理金额" in result:
        ywblje = case.split("办理的")
        if ywblje:
            ywblje = ywblje[1].replace(",","").replace("，","")
            ywblje = re.search(r'\d+', ywblje)
            if ywblje:
                result["业务办理金额"] = ywblje.group(0)

    if "余额" in case and "余额数目" in result and result["余额数目"]==None:
        ye = case.replace(",","").replace("，","")
        ye = re.search(r'(余额|余额为)(\d+)', ye)
        if ye:
            result["余额数目"] = ye.group(1)

    if "余额币种" in result:
        yebz = case.replace(")", "）").replace("(", "（")
        yebz = yebz.replace(",", "").replace("，","")
        yebz = re.search(r'(余额|余额为)\d+\（(.*?)\）', yebz)
        if yebz:
            result["余额币种"] = yebz.group(2)

    if "承担相关还款责任的业务" in result:
        tgt = re.search(r'办理的(.*?)承担相关还款责任', case)
        if tgt:
            result["承担相关还款责任的业务"] = tgt.group(1)

    if "逾期超过90天月数" in result:
        yq90 = re.search(r'其中(.*?)逾期超过90天', case)
        if yq90:
            result["逾期超过90天月数"] = yq90.group(1)
            
    return result


# 个人信用报告

!!! 更新

In [44]:
key_2_English("信息概要")

'information_summary'

In [198]:
en = ["inquiry_records",
    "institutional_inquiry_details",
    "personal_inquiry_details"]
ch = ["查询记录", "机构查询记录明细", "本人查询记录明细"]
for e,c in zip(en, ch):
    update_trans_dict(c, e)

In [200]:
grxybg_data = [
    "报告编号：2016080303000014210351", 
    "报告时间：2016-08-03 09:30:15",  # 确保分割标志是中文冒号, 其他事英文冒号
    "姓名：张三",
    "证件类型：身份证",
    "证件号码：110124197506232452",
    "已婚",
    "其他证件信息：护照G300234234/军人身份证 M09876893"
]

In [201]:
def grxybg_info_extraction(grxybg_data):
    result = {}
    for e in grxybg_data:
        e = e.replace("：", ":")
        k_v = e.split(":", 1)
        if len(k_v) == 2:
            result[k_v[0]] = k_v[1]
        elif len(k_v) == 1:
            result["婚姻状况"] = e
    return result

grxybg_result = grxybg_info_extraction(grxybg_data)
grxybg_result

{'报告编号': '2016080303000014210351',
 '报告时间': '2016-08-03 09:30:15',
 '姓名': '张三',
 '证件类型': '身份证',
 '证件号码': '110124197506232452',
 '婚姻状况': '已婚',
 '其他证件信息': '护照G300234234/军人身份证 M09876893'}

In [20]:
{k:key_2_English(k) for k,v in grxybg_result.items()}

{'报告编号': 'report_id',
 '报告时间': 'report_time',
 '姓名': 'name',
 '证件类型': 'id_type',
 '证件号码': 'id_number',
 '婚姻状况': 'marital_status',
 '其他证件信息': 'other_document_information'}

In [22]:
grxybg_info = {key_2_English(k):v for k,v in grxybg_result.items()}
grxybg_info

{'report_id': '2016080303000014210351',
 'report_time': '2016-08-03 09:30:15',
 'name': '张三',
 'id_type': '身份证',
 'id_number': '110124197506232452',
 'marital_status': '已婚',
 'other_document_information': '护照G300234234/军人身份证 M09876893'}

## == 内容样例

![个人信用报告](4_简版方案/个人信用报告.png)

In [8]:
# grxybg_data = """
# \n报告编号：2016080303000014210351 报告时间：2016-08-03 09:30:15\n姓名: 张三 证件类型: 身份证 证件号码: 110124197506232452 已婚\n其他证件信息：护照G300234234/军人身份证 M09876893\n
# """
# grxybg_data

'\n\n报告编号：2016080303000014210351 报告时间：2016-08-03 09:30:15\n姓名: 张三 证件类型: 身份证 证件号码: 110124197506232452 已婚\n其他证件信息：护照G300234234/军人身份证 M09876893\n\n'

In [9]:
# def grxybg_text_preprocess(data):
#     data = data.strip().strip("\n").strip()
#     data = data.replace("\n", "  ")
#     return data

# grxybg_data = grxybg_text_preprocess(data=grxybg_data)
# grxybg_data

'报告编号：2016080303000014210351 报告时间：2016-08-03 09:30:15  姓名: 张三 证件类型: 身份证 证件号码: 110124197506232452 已婚  其他证件信息：护照G300234234/军人身份证 M09876893'

## == 预期提取效果

整体效果（提取结果存放在字段 `info_extraction`）：
```json
{
    'header-0':{
        'header-name':'个人信用报告',
        'content':[ocr返回结果],
        'info_extraction':[result]
    }
}
```
`result` 关键字段及其中英对照：

In [15]:
# '''人工提前确定好所有关键字段'''
# grxybg_schema = []
# grxybg_schema.extend(['报告编号', '报告时间'])
# grxybg_schema.extend(['姓名', '证件类型', '证件号码', '是否已婚'])
# grxybg_schema.extend(['其他证件信息'])

# result = {k:key_2_English(k) for k in grxybg_schema}
# result

{'报告编号': 'report_id',
 '报告时间': 'report_time',
 '姓名': 'name',
 '证件类型': 'id_type',
 '证件号码': 'id_number',
 '是否已婚': 'married_or_not',
 '其他证件信息': 'other_document_information'}

In [14]:
## 翻译修正
# update_trans_dict("报告编号","report_id")
# update_trans_dict("证件类型","id_type")

## == 提取方案

1. 预处理文本
2. 人工定义所有要提取的关键字段
3. paddleNLP taskflow information extraction 信息抽取
4. 人工调整校验

## == 案例执行结果

------------------------------ 模型抽取 ----------------------------------

In [19]:
# """ 加载信息抽取模型 """
# ## 默认uie_base
# grxybg_result = extract_info_from_taskflow(task=ie, case=grxybg_data, schema=grxybg_schema)

# grxybg_result

{'报告编号': '2016080303000014210351',
 '报告时间': '2016-08-03 09:30:15',
 '姓名': '张三',
 '证件类型': '身份证',
 '证件号码': '110124197506232452',
 '是否已婚': '已婚',
 '其他证件信息': '护照G300234234'}

- `其他证件信息` 识别不完整

--------------------------------- 人工矫正 -------------------------------

In [20]:
# def grxybg_result_fixing(case, result):
#     '''
#     1. 其他证件信息
#         默认该项是最后一个item,直接包含到case最后
#     2. ... (待补充矫正)
#     '''
#     ## 其他证件信息
#     item = "其他证件信息"
#     matches = list(re.finditer(item, case))
#     if len(matches) == 0:
#         item_info = None
#     else:
#         idx = matches[0].end()
#         item_info = case[idx:]
#         item_info = re.sub(r'[\n:： ]', '', item_info)
#     result[item] = item_info

#     return result

In [21]:
# ## 人工矫正
# grxybg_result = grxybg_result_fixing(case=grxybg_data, result=grxybg_result)

# print(grxybg_data)
# ## key用英文显示
# {key_2_English(k):v for k,v in grxybg_result.items()}

报告编号：2016080303000014210351 报告时间：2016-08-03 09:30:15  姓名: 张三 证件类型: 身份证 证件号码: 110124197506232452 已婚  其他证件信息：护照G300234234/军人身份证 M09876893


{'report_id': '2016080303000014210351',
 'report_time': '2016-08-03 09:30:15',
 'name': '张三',
 'id_type': '身份证',
 'id_number': '110124197506232452',
 'married_or_not': '已婚',
 'other_document_information': '护照G300234234/军人身份证M09876893'}

# 信贷记录

## 信息概要

#### == 内容样例

![信息概要](./4_简版方案/信息概要.png)

In [59]:
xxgy_data = []
xxgy_data.append([
    ['', '资产处置信息', '垫款信息'], 
    ['账户数', '1', '3']])

xxgy_data.append([
    ['', '信用卡', '贷款', None, '其他业务'],
    [None, None, '购房', '其他', None],
    ['账户数', '8', '4', '5', '8'],
    [None, None, None, None, None],
    ['未结清/未销户账户数', '4', '2', '3', '7'],
    ['发生过逾期的账户数', '4', '2', '2', '4'],
    ['发生过90天以上逾期的账户数', '4', '1', '1', '1']])

xxgy_data.append([
    ['', '为个人', '为企业'], 
    ['相关还款责任账户数', '1', '3']])

xxgy_data

[[['', '资产处置信息', '垫款信息'], ['账户数', '1', '3']],
 [['', '信用卡', '贷款', None, '其他业务'],
  [None, None, '购房', '其他', None],
  ['账户数', '8', '4', '5', '8'],
  [None, None, None, None, None],
  ['未结清/未销户账户数', '4', '2', '3', '7'],
  ['发生过逾期的账户数', '4', '2', '2', '4'],
  ['发生过90天以上逾期的账户数', '4', '1', '1', '1']],
 [['', '为个人', '为企业'], ['相关还款责任账户数', '1', '3']]]

#### == 预期提取效果

整体上（提取结果存放在字段 `info_extraction`）：
```json
'header-1-0':{
    'header-name':'信息概要',
    'content':[ocr返回结果],
    'info_extraction':[table_1_info, table_2_info, table_3_info]
}
```
`table_i_info` 样式如下：

一行一行读，嵌套标题会被展开
```json
{
    row_name_1:{
        col_name_1:value, col_name_2:value, ...}
    },
    row_name_2:{
        col_name_1:value, col_name_2:value, ...}
    },
    ...
}
```

#### == 提取方案

1. 预处理
    - 删空值行
    - 默认第一列为行名
    - 无数字的前i(<=2)行为列名
2. 一行一行按规则提取

#### == 案例执行结果

------------------------------ 表格拆解函数 --------------------------------

In [60]:
def drop_None_row(table:list[list]):
    '''
    drop a row if its all None
    '''
    return [row for row in table if any(e is not None for e in row)]

def is_number(s:str):
    ## 不包含空值和None
    if s is None or s == '':
        return False
    ## 其他值判断是否为纯数字
    try:
        float(s)
        return True
    except ValueError:
        return False
        
def get_col_name_rows(table:list[list]):
    '''
    find first n rows that have no numbers
    '''
    for i in range(len(table)):
        row = table[i]
        flag = not any(is_number(e) for e in row)  ## 是否为标题行
        if flag == False:
            break
    return table[:i]

def is_empty(s:str):
    if s is None or s == '':
        return True
    else:
        return False
        
def combine_col_names(col_name_rows:list[list]):
    '''
    返回：行名、列名、数据
    
    逻辑：
        1. 默认合并单元格的信息在左上格
        2. 从末列向前：
            a. 空值向上传递，遇到非空值被覆盖，继续向上转b
            b. 非空值向上传递，(遇到空值，找该空值左边第一个非空值合并)，继续向上重复
    
    表格转置后可用在合并行名上
    '''
    R = len(col_name_rows)
    C = len(col_name_rows[0])

    col_names = []
    for c in range(C-1, -1, -1):
        col_name = col_name_rows[R-1][c]
        
        empty = False  ## whether col_name is empty now
        if is_empty(col_name):
            empty = True
            
        for r in range(R-2, -1, -1):
            up = col_name_rows[r][c]
            
            if empty:
                if is_empty(up): ## 空值连续向上
                    continue
                else:
                    col_name = up  ## 空值被非空值覆盖
                    empty = False
            else:
                if is_empty(up):
                    ## 非空值向上遇到空值, 找空值左边第一个非空值作为母标题
                    left = ''
                    for c_temp in range(c-1, -1, -1):
                        left = col_name_rows[r][c_temp]
                        if not is_empty(left):
                            col_name = left + "_" + col_name
                            break
                else:
                    col_name = up + "_" + col_name

        if not is_empty(col_name):  ## 存入非空列名
            col_names.append(col_name)

    return col_names[::-1]
                
                
def xxgy_preprocess_table(table:list[list]):  ## 信息概要
    '''
    忽略首格[0][0]（默认为空）
    默认行名在第一列
    
    可修改复用在标准表格上：
        列名在左 + 行名在上 + 数字数据在中间
    '''
    ## 删全为None的行
    table = drop_None_row(table)
    R = len(table)
    C = len(table[0])
    # print(table)

    ## 找'列名'的行:
    col_name_rows = get_col_name_rows(table)
    num_col_name_rows = len(col_name_rows)  ## 前几行是标题行
    # print(num_col_name_rows)
    
    ## 提取列名
    col_names = combine_col_names(col_name_rows)

    ## 提取行名, 默认第一列
    row_names = []
    for r in range(num_col_name_rows, R):
        row_names.append(table[r][0])

    ## 提取数据:
    ##     table里倒数 len(row_names)行, 和倒数 len(col_names)列
    data = []
    for r in range(R-len(row_names), R):
        data_row = []
        for c in range(C-len(col_names), C):
            data_row.append(table[r][c])
        data.append(data_row)

    return row_names, col_names, data    

------------------------------ 表格拆解结果 ------------------------------

In [61]:
table_1_result = xxgy_preprocess_table(xxgy_data[0])
table_1_result

(['账户数'], ['资产处置信息', '垫款信息'], [['1', '3']])

In [62]:
table_2_result = xxgy_preprocess_table(xxgy_data[1])
table_2_result

(['账户数', '未结清/未销户账户数', '发生过逾期的账户数', '发生过90天以上逾期的账户数'],
 ['信用卡', '贷款_购房', '贷款_其他', '其他业务'],
 [['8', '4', '5', '8'],
  ['4', '2', '3', '7'],
  ['4', '2', '2', '4'],
  ['4', '1', '1', '1']])

In [63]:
table_3_result = xxgy_preprocess_table(xxgy_data[2])
table_3_result

(['相关还款责任账户数'], ['为个人', '为企业'], [['1', '3']])

------------------------------------ 信息抽取结果 -----------------------------------

In [64]:
def xxgy_table_result_to_json(row_names, col_names, data):
    table_dict = {}
    for r in range(len(row_names)):
        r_name = row_names[r]
        table_dict[r_name] = {}
        for c in range(len(col_names)):
            c_name = col_names[c]
            table_dict[r_name][c_name]=data[r][c]
    return table_dict  

In [65]:
table_1_info = xxgy_table_result_to_json(row_names=key_list_2_English(table_1_result[0]),
                                         col_names=key_list_2_English(table_1_result[1]),
                                        data=key_list_2_English(table_1_result[2]))
display(pd.DataFrame(xxgy_data[0]))
table_1_info

,0,1,2
0,,资产处置信息,垫款信息
1,账户数,1,3


{'number_of_accounts': {'asset_disposal_information': '1',
  'overdraft_information': '3'}}

In [66]:
table_2_info = xxgy_table_result_to_json(row_names=key_list_2_English(table_2_result[0]),
                                         col_names=key_list_2_English(table_2_result[1]),
                                        data=key_list_2_English(table_2_result[2]))
display(pd.DataFrame(xxgy_data[1]))
table_2_info

,0,1,2,3,4
0,,信用卡,贷款,None,其他业务
1,None,None,购房,其他,None
2,账户数,8,4,5,8
3,None,None,None,None,None
4,未结清/未销户账户数,4,2,3,7
5,发生过逾期的账户数,4,2,2,4
6,发生过90天以上逾期的账户数,4,1,1,1


{'number_of_accounts': {'credit_cards': '8',
  'loan__buy_home': '4',
  'loan__other': '5',
  'other_business': '8'},
 'number_of_openuncancelled_accounts': {'credit_cards': '4',
  'loan__buy_home': '2',
  'loan__other': '3',
  'other_business': '7'},
 'number_of_accounts_with_records_of_being_overdue': {'credit_cards': '4',
  'loan__buy_home': '2',
  'loan__other': '2',
  'other_business': '4'},
 'number_of_accounts_with_records_of_being_overdue_for_over_90_days': {'credit_cards': '4',
  'loan__buy_home': '1',
  'loan__other': '1',
  'other_business': '1'}}

In [67]:
table_3_info = xxgy_table_result_to_json(row_names=key_list_2_English(table_3_result[0]),
                                         col_names=key_list_2_English(table_3_result[1]),
                                        data=key_list_2_English(table_3_result[2]))
display(pd.DataFrame(xxgy_data[2]))
table_3_info

,0,1,2
0,,为个人,为企业
1,相关还款责任账户数,1,3


{'number_of_related_repayment_liability_accounts': {'for_individuals': '1',
  'for_business': '3'}}

In [68]:
[table_1_info, table_2_info, table_3_info]

[{'number_of_accounts': {'asset_disposal_information': '1',
   'overdraft_information': '3'}},
 {'number_of_accounts': {'credit_cards': '8',
   'loan__buy_home': '4',
   'loan__other': '5',
   'other_business': '8'},
  'number_of_openuncancelled_accounts': {'credit_cards': '4',
   'loan__buy_home': '2',
   'loan__other': '3',
   'other_business': '7'},
  'number_of_accounts_with_records_of_being_overdue': {'credit_cards': '4',
   'loan__buy_home': '2',
   'loan__other': '2',
   'other_business': '4'},
  'number_of_accounts_with_records_of_being_overdue_for_over_90_days': {'credit_cards': '4',
   'loan__buy_home': '1',
   'loan__other': '1',
   'other_business': '1'}},
 {'number_of_related_repayment_liability_accounts': {'for_individuals': '1',
   'for_business': '3'}}]

In [79]:
a = table_1_result[0]
a.extend(table_1_result[1])
a.extend(table_2_result[0])
a.extend(table_2_result[1])
a.extend(table_3_result[0])
a.extend(table_3_result[1])
{k:key_2_English(k) for k in a}

{'账户数': 'number_of_accounts',
 '资产处置信息': 'asset_disposal_information',
 '垫款信息': 'overdraft_information',
 '未结清/未销户账户数': 'number_of_openuncancelled_accounts',
 '发生过逾期的账户数': 'number_of_accounts_with_records_of_being_overdue',
 '发生过90天以上逾期的账户数': 'number_of_accounts_with_records_of_being_overdue_for_over_90_days',
 '信用卡': 'credit_cards',
 '贷款_购房': 'loan__buy_home',
 '贷款_其他': 'loan__other',
 '其他业务': 'other_business',
 '相关还款责任账户数': 'number_of_related_repayment_liability_accounts',
 '为个人': 'for_individuals',
 '为企业': 'for_business'}

#### == 备注

- !! 具体操作需根据ocr返回结果修改
- 翻译矫正

### 资产处置信息
简写 -- 具体思路同 `信用卡 -- 逾期贷记卡`

#### -- 定义关键字段 

In [141]:
'''关键字段需人工提前确定'''
zcczxx_schema = ["债权接收日期", "债权接收单位", "债权接收金额"]
zcczxx_schema.extend(["截至日期", "余额数目", "最近还款日期"])

{k:key_2_English(k) for k in zcczxx_schema}

{'债权接收日期': 'date_of_claim_receipt',
 '债权接收单位': 'entity_receiving_the_claim',
 '债权接收金额': 'amount_of_claim_received',
 '截至日期': 'reference_date',
 '余额数目': 'remaining_balance',
 '最近还款日期': 'last_repayment_date'}

In [24]:
## 翻译修正
# update_trans_dict('债权接收日期', 'date_of_claim_receipt')
# update_trans_dict('债权接收单位', 'entity_receiving_the_claim')
# update_trans_dict('债权接收金额', 'amount_of_claim_received')
# update_trans_dict('余额截至日期', 'balance_reference_date')
# update_trans_dict('余额数目', 'remaining_balance')
# update_trans_dict('最近还款日期', 'last_repayment_date')

In [34]:
a = ["信贷记录", "信息概要", "资产处置信息"]
{k:key_2_English(k) for k in a}

{'信贷记录': 'credit_records',
 '信息概要': 'information_summary',
 '资产处置信息': 'asset_disposal_information'}

#### -- 预处理数据

In [24]:
## 预期ocr结果
# zcczxx_data = """
# \n1.2012年03月12日，东方资产管理公司接收债权，金额为11,000,000。截至 2014年11月11日，余额为920,000，最近一次\n还款日期为2015年12月11日。\n
# """
# zcczxx_data

'\n\n1.2012年03月12日，东方资产管理公司接收债权，金额为11,000,000。截至 2014年11月11日，余额为920,000，最近一次\n还款日期为2015年12月11日。\n\n'

In [26]:
## 每个条目为一段
# zcczxx_cases = split_cases_by_idx_from_paragraph(zcczxx_data)
# zcczxx_cases

['2012年03月12日，东方资产管理公司接收债权，金额为11,000,000。截至 2014年11月11日，余额为920,000，最近一次还款日期为2015年12月11日。']

!!! 更新

In [143]:
## 更新为
zcczxx_data = []
zcczxx_data.append("2012年03月12日，东方资产管理公司接收债权，金额为11,000,000。截至2014年11月11日，余额为920,000，最近一次还款日期为2015年12月11日。")
zcczxx_data

['2012年03月12日，东方资产管理公司接收债权，金额为11,000,000。截至2014年11月11日，余额为920,000，最近一次还款日期为2015年12月11日。']

#### -- 信息抽取结果

In [144]:
zcczxx_results = []
for case in zcczxx_cases:
    ## 模型提取
    result = extract_info_from_taskflow(case=case, schema=zcczxx_schema)
    zcczxx_results.append(result)

In [147]:
i = 0
print(zcczxx_data[i])
display(pd.DataFrame(list(zcczxx_results[i].items()), columns=['key', 'value']))

2012年03月12日，东方资产管理公司接收债权，金额为11,000,000。截至2014年11月11日，余额为920,000，最近一次还款日期为2015年12月11日。


,key,value
0,债权接收日期,2012年03月12日
1,债权接收单位,东方资产管理公司
2,债权接收金额,"11,000,000"
3,截至日期,2014年11月11日
4,余额数目,"920,000"
5,最近还款日期,2015年12月11日


In [146]:
zcczxx_results_fixed = []
for case, result in zip(zcczxx_data, zcczxx_results):
    result = text_result_fixing(case, result)
    zcczxx_results_fixed.append(result)

In [148]:
for case, result in zip(zcczxx_data, zcczxx_results_fixed):
    print(zcczxx_data[i])
    display(pd.DataFrame(list(zcczxx_results_fixed[i].items()), columns=['key', 'value']))

2012年03月12日，东方资产管理公司接收债权，金额为11,000,000。截至2014年11月11日，余额为920,000，最近一次还款日期为2015年12月11日。


,key,value
0,债权接收日期,2012年03月12日
1,债权接收单位,东方资产管理公司
2,债权接收金额,"11,000,000"
3,截至日期,2014年11月11日
4,余额数目,"920,000"
5,最近还款日期,2015年12月11日


#### -- en

In [153]:
zcczxx_results_fixed_en = []
for result in zcczxx_results_fixed:
    result = {key_2_English(k):v for k,v in result.items()}
    zcczxx_results_fixed_en.append(result)

zcczxx_results_fixed_en

[{'date_of_claim_receipt': '2012年03月12日',
  'entity_receiving_the_claim': '东方资产管理公司',
  'amount_of_claim_received': '11,000,000',
  'reference_date': '2014年11月11日',
  'remaining_balance': '920,000',
  'last_repayment_date': '2015年12月11日'}]

### 垫款信息 
简写 -- 具体思路同 `信用卡 -- 逾期贷记卡`

#### -- 定义关键字段

In [50]:
update_trans_dict("垫款信息", "overdraft_information")

In [160]:
'''关键字段需人工提前确定'''
dkxx_schema = ["累计代偿金额开始日期", "累计代偿金额所属单位", "累计代偿金额数目"]
dkxx_schema.extend(["结清日期"])
dkxx_schema.extend(["余额截至日期", "余额数目", "最近还款日期"])

{k:key_2_English(k) for k in dkxx_schema}

{'累计代偿金额开始日期': 'start_date_of_accumulated_indemnified_amount',
 '累计代偿金额所属单位': 'entity_responsible_for_accumulated_indemnified_amount',
 '累计代偿金额数目': 'accumulated_indemnified_amount',
 '结清日期': 'date_of_settlement',
 '余额截至日期': 'balance_reference_date',
 '余额数目': 'remaining_balance',
 '最近还款日期': 'last_repayment_date'}

#### -- 预处理数据

In [33]:
## 预期ocr结果
# dkxx_data = """
# \n1.2012年02月02日以来中国人寿保险公司累计代偿金额80，000。截至2014年09月9日，余额70,000，最近一次还款日期为\n2014年10月10日。\n2.2013年02月02日以来富登融资租赁担保公司累计代偿金额400,000。2016年1月已结清。\n
# """
# dkxx_data

'\n\n1.2012年02月02日以来中国人寿保险公司累计代偿金额80，000。截至2014年09月9日，余额70,000，最近一次还款日期为\n2014年10月10日。\n2.2013年02月02日以来富登融资租赁担保公司累计代偿金额400,000。2016年1月已结清。\n\n'

In [34]:
## 每个条目为一段
# dkxx_cases = split_cases_by_idx_from_paragraph(dkxx_data)
# dkxx_cases

['2012年02月02日以来中国人寿保险公司累计代偿金额80，000。截至2014年09月9日，余额70,000，最近一次还款日期为2014年10月10日。',
 '2013年02月02日以来富登融资租赁担保公司累计代偿金额400,000。2016年1月已结清。']

In [156]:
dkxx_data = []
dkxx_data.append("2012年02月02日以来中国人寿保险公司累计代偿金额80，000。截至2014年09月9日，余额70,000，最近一次还款日期为2014年10月10日。")
dkxx_data.append("2013年02月02日以来富登融资租赁担保公司累计代偿金额400,000。2016年1月已结清。")
dkxx_data

['2012年02月02日以来中国人寿保险公司累计代偿金额80，000。截至2014年09月9日，余额70,000，最近一次还款日期为2014年10月10日。',
 '2013年02月02日以来富登融资租赁担保公司累计代偿金额400,000。2016年1月已结清。']

#### -- 信息抽取结果

In [69]:
# """ 人工矫正抽取信息 """
# def dkxx_result_fixing(case, result):
#     '''
#     矫正逻辑：
#         1. 若含有“已结清”：
#                 是否已结清：是
#             否则
#                 结清日期：None
#         2. 若不含有“截至”：
#                 截至日期：None
#         3. 若不含有“余额”：
#                 余额数目: None
#         4. 若不含有“还款日期”: 
#                 最近还款日期：None
#     '''
#     case = case.replace(" ", "")  ## 删空格
    
#     if '已结清' in case:
#         result["是否已结清"] = '是'
#     else:
#         result["结清日期"] = None

#     if '截至' not in case:
#         result["余额截至日期"] = None
        
#     if "余额" not in case:
#         result["余额数目"] = None

#     if '还款日期' not in case:
#         result["最近还款日期"] = None

#     return result


In [161]:
dkxx_results=[]
dkxx_results_fixed = []
dkxx_results_fixed_en = []

for case in dkxx_data:
    ## 模型提取
    result = extract_info_from_taskflow(case=case, schema=dkxx_schema)
    dkxx_results.append(result)
    ## 结果展示
    print("-----------")
    print(case)
    display(pd.DataFrame(list(result.items()), columns=['key', 'value']))
    ## 结果矫正
    result = text_result_fixing(case, result)
    display(pd.DataFrame(list(result.items()), columns=['key', 'value']))
    dkxx_results_fixed.append(result)
    ## key值转英文
    result = {key_2_English(k):v for k,v in result.items()}
    # display(pd.DataFrame(list(result.items()), columns=['key', 'value']))
    dkxx_results_fixed_en.append(result)

    results.append(result)

-----------
2012年02月02日以来中国人寿保险公司累计代偿金额80，000。截至2014年09月9日，余额70,000，最近一次还款日期为2014年10月10日。


,key,value
0,累计代偿金额开始日期,2012年02月02日
1,累计代偿金额所属单位,中国人寿保险公司
2,累计代偿金额数目,80，000
3,结清日期,2014年09月9日
4,余额截至日期,2014年09月9日
5,余额数目,"70,000"
6,最近还款日期,2014年10月10日


,key,value
0,累计代偿金额开始日期,2012年02月02日
1,累计代偿金额所属单位,中国人寿保险公司
2,累计代偿金额数目,80，000
3,结清日期,None
4,余额截至日期,2014年09月9日
5,余额数目,"70,000"
6,最近还款日期,2014年10月10日


,key,value
0,start_date_of_accumulated_indemnified_amount,2012年02月02日
1,entity_responsible_for_accumulated_indemnified...,中国人寿保险公司
2,accumulated_indemnified_amount,80，000
3,date_of_settlement,None
4,balance_reference_date,2014年09月9日
5,remaining_balance,"70,000"
6,last_repayment_date,2014年10月10日


-----------
2013年02月02日以来富登融资租赁担保公司累计代偿金额400,000。2016年1月已结清。


,key,value
0,累计代偿金额开始日期,2013年02月02日
1,累计代偿金额所属单位,富登融资租赁担保公司
2,累计代偿金额数目,"400,000"
3,结清日期,2016年1月
4,余额截至日期,2013年02月02日
5,余额数目,"400,000"
6,最近还款日期,2013年02月02日


,key,value
0,累计代偿金额开始日期,2013年02月02日
1,累计代偿金额所属单位,富登融资租赁担保公司
2,累计代偿金额数目,"400,000"
3,结清日期,2016年1月
4,余额截至日期,None
5,余额数目,None
6,最近还款日期,None


,key,value
0,start_date_of_accumulated_indemnified_amount,2013年02月02日
1,entity_responsible_for_accumulated_indemnified...,富登融资租赁担保公司
2,accumulated_indemnified_amount,"400,000"
3,date_of_settlement,2016年1月
4,balance_reference_date,None
5,remaining_balance,None
6,last_repayment_date,None


#### -- en

In [162]:
dkxx_results_fixed_en

[{'start_date_of_accumulated_indemnified_amount': '2012年02月02日',
  'entity_responsible_for_accumulated_indemnified_amount': '中国人寿保险公司',
  'accumulated_indemnified_amount': '80，000',
  'date_of_settlement': None,
  'balance_reference_date': '2014年09月9日',
  'remaining_balance': '70,000',
  'last_repayment_date': '2014年10月10日'},
 {'start_date_of_accumulated_indemnified_amount': '2013年02月02日',
  'entity_responsible_for_accumulated_indemnified_amount': '富登融资租赁担保公司',
  'accumulated_indemnified_amount': '400,000',
  'date_of_settlement': '2016年1月',
  'balance_reference_date': None,
  'remaining_balance': None,
  'last_repayment_date': None}]

## 信用卡

- !! 下面三个小块可能共用矫正函数

### 发生过逾期的贷记卡账户明细如下：

#### == 内容样例

![发生过逾期的贷记卡账户明细](./4_简版方案/发生过逾期的贷记卡账户明细.png)

In [6]:
# xyk_yq_data = """
# \n1. 2004 年 8 月 30 日中国建设银行北京分行发放的贷记卡（人民币账户，卡片尾号：0001）。截至 2010 年 10 月，信用额度\n10,000，已变成呆账，余额500。\n2. 2004年8月30日中国工商银行北京分行发放的贷记卡（人民币账户，卡片尾号：0002）。截至2016年7月，信用额度50,000，\n余额5,000（含未出单的大额专项分期余额4,000），当前有逾期。最近5年内有11个月处于逾期状态，其中5个月逾期超\n过90天。\n3. 2010年4月1日中国民生银行信用卡中心发放的贷记卡（人民币账户，卡片尾号：0003），2015年12月销户。最近5年内\n有7个月处于逾期状态，其中3个月逾期超过90天。\n2014年3月，该机构声明：该客户委托XX公司偿还贷款，因开发公司不按时还款导致出现多次逾期。\n
# """
# xyk_yq_data

'\n\n1. 2004 年 8 月 30 日中国建设银行北京分行发放的贷记卡（人民币账户，卡片尾号：0001）。截至 2010 年 10 月，信用额度\n10,000，已变成呆账，余额500。\n2. 2004年8月30日中国工商银行北京分行发放的贷记卡（人民币账户，卡片尾号：0002）。截至2016年7月，信用额度50,000，\n余额5,000（含未出单的大额专项分期余额4,000），当前有逾期。最近5年内有11个月处于逾期状态，其中5个月逾期超\n过90天。\n3. 2010年4月1日中国民生银行信用卡中心发放的贷记卡（人民币账户，卡片尾号：0003），2015年12月销户。最近5年内\n有7个月处于逾期状态，其中3个月逾期超过90天。\n2014年3月，该机构声明：该客户委托XX公司偿还贷款，因开发公司不按时还款导致出现多次逾期。\n\n'

In [8]:
# """预处理分割data"""
# cases = split_cases_by_idx_from_paragraph(data=xyk_yq_data)
# case_1, case_2, case_3 = cases[0], cases[1], cases[2]
# print(f"case_1: {case_1.replace(' ', '')} \n")
# print(f"case_2: {case_2.replace(' ', '')} \n")
# print(f"case_3: {case_3.replace(' ', '')} \n")

case_1: 2004年8月30日中国建设银行北京分行发放的贷记卡（人民币账户，卡片尾号：0001）。截至2010年10月，信用额度10,000，已变成呆账，余额500。 

case_2: 2004年8月30日中国工商银行北京分行发放的贷记卡（人民币账户，卡片尾号：0002）。截至2016年7月，信用额度50,000，余额5,000（含未出单的大额专项分期余额4,000），当前有逾期。最近5年内有11个月处于逾期状态，其中5个月逾期超过90天。 

case_3: 2010年4月1日中国民生银行信用卡中心发放的贷记卡（人民币账户，卡片尾号：0003），2015年12月销户。最近5年内有7个月处于逾期状态，其中3个月逾期超过90天。2014年3月，该机构声明：该客户委托XX公司偿还贷款，因开发公司不按时还款导致出现多次逾期。 



!!! 更新

In [163]:
## 更新
xyk_yq_data = [
    "2004年8月30日中国建设银行北京分行发放的贷记卡（人民币账户，卡片尾号：0001）。截至2010年10月，信用额度10,000，已变成呆账，余额500。",
    "2004年8月30日中国工商银行北京分行发放的贷记卡（人民币账户，卡片尾号：0002）。截至2016年7月，信用额度50,000，余额5,000（含未出单的大额专项分期余额4,000），当前有逾期。最近5年内有11个月处于逾期状态，其中5个月逾期超过90天。",
    "2010年4月1日中国民生银行信用卡中心发放的贷记卡（人民币账户，卡片尾号：0003），2015年12月销户。最近5年内有7个月处于逾期状态，其中3个月逾期超过90天。",
    "2014年3月，该机构声明：该客户委托XX公司偿还贷款，因开发公司不按时还款导致出现多次逾期。"
    
]

case_1, case_2, case_3, case_4 = xyk_yq_data
xyk_yq_data

['2004年8月30日中国建设银行北京分行发放的贷记卡（人民币账户，卡片尾号：0001）。截至2010年10月，信用额度10,000，已变成呆账，余额500。',
 '2004年8月30日中国工商银行北京分行发放的贷记卡（人民币账户，卡片尾号：0002）。截至2016年7月，信用额度50,000，余额5,000（含未出单的大额专项分期余额4,000），当前有逾期。最近5年内有11个月处于逾期状态，其中5个月逾期超过90天。',
 '2010年4月1日中国民生银行信用卡中心发放的贷记卡（人民币账户，卡片尾号：0003），2015年12月销户。最近5年内有7个月处于逾期状态，其中3个月逾期超过90天。',
 '2014年3月，该机构声明：该客户委托XX公司偿还贷款，因开发公司不按时还款导致出现多次逾期。']

#### == 预期提取效果

整体（提取结果存放在字段 `info_extraction`）：
```json
{
    'header-1-1-0':{
        'header-name':'发生过逾期的贷记卡账户明细',
        'content': [ocr返回结果]
        'info_extraction':[case_0_info, case_1_info, case_2_info]
    }
}
```
`case_i_info` 关键字段及其中英对照：

In [173]:
'''人工提前定好schema内容'''
xyk_yq_schema = ['发卡日期', '发卡机构','账户币种','卡片尾号','销户日期']
xyk_yq_schema.extend(['信用额度截至日期','信用额度', '是否为呆账','总余额', '未出单的大额专项分期余额', '当前有无逾期'])
xyk_yq_schema.extend(['近5年处于逾期状态月数','逾期超过90天月数'])  

case_i_info = {k:key_2_English(k) for k in xyk_yq_schema}
case_i_info

{'发卡日期': 'card_issuing_date',
 '发卡机构': 'card_issuer',
 '账户币种': 'account_currency',
 '卡片尾号': 'card_suffix',
 '销户日期': 'account_canceling_date',
 '信用额度截至日期': 'reference_date',
 '信用额度': 'credit_limit',
 '是否为呆账': 'whether_it_is_a_bad_debt',
 '总余额': 'total_balance',
 '未出单的大额专项分期余额': 'large_amount_of_unissued_special_installment_balance',
 '当前有无逾期': 'is_currently_overdue',
 '近5年处于逾期状态月数': 'number_of_months_overdue_in_the_last_5_years',
 '逾期超过90天月数': 'number_of_months_overdue_for_more_than_90_days'}

In [36]:
print(key_2_English('信用卡'))
print(key_2_English('发生过逾期的贷记卡账户明细如下'))

credit_cards
credit_card_account_details_with_overdue_records


In [35]:
update_trans_dict("发生过逾期的贷记卡账户明细如下", "credit_card_account_details_with_overdue_records")

In [95]:
"""手动矫正翻译"""
# update_trans_dict("销户日期", "account_canceling_date")
# update_trans_dict("卡片尾号", "card_suffix")
# update_trans_dict("信用额度截至日期", "reference_date")
# update_trans_dict("近5年处于逾期状态月数", "number_of_months_overdue_in_the_last_5_years")
# update_trans_dict("逾期超过90天月数", "number_of_months_overdue_for_more_than_90_days")
# update_trans_dict("发卡机构", "card_issuer")
# update_trans_dict("发卡日期", "card_issue_date")

#### == 提取方案

1. 文本预处理，手动分割段落
1. 人工定义提取关键字段 `djkyq_schema`
   - 待补充 （请业务人员给出建议）
2. paddleNLP taskflow information extraction 用小模型进行字段信息抽取
3. 后续人工用规则矫正
    - 详见下文 `人工修正函数`
    - !! 待补充（请测试/业务人员给出建议）
    

#### == 案例执行结果

------------------------------ 模型抽取 ----------------------------------

In [174]:
xyk_yq_results = []
for case in xyk_yq_data:
    result = extract_info_from_taskflow(case=case, schema=xyk_yq_schema)
    xyk_yq_results.append(result)

In [175]:
i = 0
print(xyk_yq_data[i])
display(pd.DataFrame(list(xyk_yq_results[i].items()), columns=['key', 'value']))

2004年8月30日中国建设银行北京分行发放的贷记卡（人民币账户，卡片尾号：0001）。截至2010年10月，信用额度10,000，已变成呆账，余额500。


,key,value
0,发卡日期,2004年8月30日
1,发卡机构,中国建设银行北京分行
2,账户币种,人民币
3,卡片尾号,0001
4,销户日期,None
5,信用额度截至日期,2010年10月
6,信用额度,"10,000"
7,是否为呆账,None
8,总余额,500
9,未出单的大额专项分期余额,None


- 问题1：无法识别`是否变成呆账`

In [176]:
i = 1
print(xyk_yq_data[i])
display(pd.DataFrame(list(xyk_yq_results[i].items()), columns=['key', 'value']))

2004年8月30日中国工商银行北京分行发放的贷记卡（人民币账户，卡片尾号：0002）。截至2016年7月，信用额度50,000，余额5,000（含未出单的大额专项分期余额4,000），当前有逾期。最近5年内有11个月处于逾期状态，其中5个月逾期超过90天。


,key,value
0,发卡日期,2004年8月30日
1,发卡机构,中国工商银行北京分行
2,账户币种,人民币
3,卡片尾号,0002
4,销户日期,None
5,信用额度截至日期,2016年7月
6,信用额度,"50,000"
7,是否为呆账,None
8,总余额,"5,000"
9,未出单的大额专项分期余额,"4,000"


- 问题2：无法识别 `当前有无逾期`
- 问题3：无法识别 `预期超过90天月数`

In [24]:
# case_2_1 = "2.1. 2004 年 8 月 30 日中国工商银行北京分行发放的贷记卡（人民币账户，卡片尾号：0002）。截至 2016 年 7 月，信用额度 50,000，余额 5,000（含未出单的大额专项分期余额 4,000），当前有逾期。最近 6 年内有 11 个月处于逾期状态，其中 5 个月逾期超过 80 天。"
# print(case_2_1)
# result_2_1 = extract_info_from_taskflow(case=case_2_1, schema=xyk_yq_schema)
# result_2_1

- 忽略 -- 问题2：识别错误（数据内改 `5年` 为 `6年`，改 `90天` 为 `80天`）

In [177]:
i = 2
print(xyk_yq_data[i])
display(pd.DataFrame(list(xyk_yq_results[i].items()), columns=['key', 'value']))

2010年4月1日中国民生银行信用卡中心发放的贷记卡（人民币账户，卡片尾号：0003），2015年12月销户。最近5年内有7个月处于逾期状态，其中3个月逾期超过90天。


,key,value
0,发卡日期,2010年4月1日
1,发卡机构,中国民生银行信用卡中心
2,账户币种,人民币
3,卡片尾号,0003
4,销户日期,2015年12月
5,信用额度截至日期,None
6,信用额度,None
7,是否为呆账,None
8,总余额,None
9,未出单的大额专项分期余额,None


In [178]:
i = 3
print(xyk_yq_data[i])
display(pd.DataFrame(list(xyk_yq_results[i].items()), columns=['key', 'value']))

2014年3月，该机构声明：该客户委托XX公司偿还贷款，因开发公司不按时还款导致出现多次逾期。


,key,value
0,发卡日期,None
1,发卡机构,None
2,账户币种,None
3,卡片尾号,None
4,销户日期,None
5,信用额度截至日期,None
6,信用额度,None
7,是否为呆账,None
8,总余额,None
9,未出单的大额专项分期余额,None


--------------------------------- 人工矫正 -------------------------------

In [26]:
# """ 定义人工修正函数 """
# def xyk_yq_result_fixing(case, result):
#     '''
#     修正逻辑：
#         1. 若含有 `呆账`, 
#                 是否变成呆账: 是
#         2. 若含有‘当前有逾期’：
#                 当前有无逾期：是
#         3. 逾期超过90天月数：
#                 规则匹配：其中？？？逾期超过90天
#     '''
#     case = case.replace(" ", "")  ## 删空格
    
#     if '呆账' in case:
#         result["是否变成呆账"] = '是'

#     # if '近5年内' not in case:
#     #     result["近5年处于逾期状态月数"] = None
#     # if '超过90天' not in case:
#     #     result["逾期超过90天月数"] = None

#     if "当前有逾期" in case:
#         result["当前有无逾期"] = '是'

#     yq90 = re.search(r'其中(.*?)逾期超过90天', case)
#     if yq90:
#         result["逾期超过90天月数"] = yq90.group(1)
        
#     return result

In [179]:
xyk_yq_results_fixed = []
for case, result in zip(xyk_yq_data, xyk_yq_results):
    result = text_result_fixing(case, result)
    xyk_yq_results_fixed.append(result)

------------------------------ 最终效果展示 -----------------------------

In [181]:
for case, result in zip(xyk_yq_data, xyk_yq_results_fixed):
    print("--------------")
    print(case)
    display(pd.DataFrame(list(result.items()), columns=['key', 'value']))

--------------
2004年8月30日中国建设银行北京分行发放的贷记卡（人民币账户，卡片尾号：0001）。截至2010年10月，信用额度10,000，已变成呆账，余额500。


,key,value
0,发卡日期,2004年8月30日
1,发卡机构,中国建设银行北京分行
2,账户币种,人民币
3,卡片尾号,0001
4,销户日期,None
5,信用额度截至日期,2010年10月
6,信用额度,"10,000"
7,是否为呆账,是
8,总余额,500
9,未出单的大额专项分期余额,None


--------------
2004年8月30日中国工商银行北京分行发放的贷记卡（人民币账户，卡片尾号：0002）。截至2016年7月，信用额度50,000，余额5,000（含未出单的大额专项分期余额4,000），当前有逾期。最近5年内有11个月处于逾期状态，其中5个月逾期超过90天。


,key,value
0,发卡日期,2004年8月30日
1,发卡机构,中国工商银行北京分行
2,账户币种,人民币
3,卡片尾号,0002
4,销户日期,None
5,信用额度截至日期,2016年7月
6,信用额度,"50,000"
7,是否为呆账,None
8,总余额,"5,000"
9,未出单的大额专项分期余额,"4,000"


--------------
2010年4月1日中国民生银行信用卡中心发放的贷记卡（人民币账户，卡片尾号：0003），2015年12月销户。最近5年内有7个月处于逾期状态，其中3个月逾期超过90天。


,key,value
0,发卡日期,2010年4月1日
1,发卡机构,中国民生银行信用卡中心
2,账户币种,人民币
3,卡片尾号,0003
4,销户日期,2015年12月
5,信用额度截至日期,None
6,信用额度,None
7,是否为呆账,None
8,总余额,None
9,未出单的大额专项分期余额,None


--------------
2014年3月，该机构声明：该客户委托XX公司偿还贷款，因开发公司不按时还款导致出现多次逾期。


,key,value
0,发卡日期,None
1,发卡机构,None
2,账户币种,None
3,卡片尾号,None
4,销户日期,None
5,信用额度截至日期,None
6,信用额度,None
7,是否为呆账,None
8,总余额,None
9,未出单的大额专项分期余额,None


In [183]:
xyk_yq_results_en = []
for r in xyk_yq_results:
    r = {key_2_English(k):v for k,v in r.items()}
    xyk_yq_results_en.append(r)

xyk_yq_results_en

[{'card_issuing_date': '2004年8月30日',
  'card_issuer': '中国建设银行北京分行',
  'account_currency': '人民币',
  'card_suffix': '0001',
  'account_canceling_date': None,
  'reference_date': '2010年10月',
  'credit_limit': '10,000',
  'whether_it_is_a_bad_debt': '是',
  'total_balance': '500',
  'large_amount_of_unissued_special_installment_balance': None,
  'is_currently_overdue': None,
  'number_of_months_overdue_in_the_last_5_years': None,
  'number_of_months_overdue_for_more_than_90_days': None},
 {'card_issuing_date': '2004年8月30日',
  'card_issuer': '中国工商银行北京分行',
  'account_currency': '人民币',
  'card_suffix': '0002',
  'account_canceling_date': None,
  'reference_date': '2016年7月',
  'credit_limit': '50,000',
  'whether_it_is_a_bad_debt': None,
  'total_balance': '5,000',
  'large_amount_of_unissued_special_installment_balance': '4,000',
  'is_currently_overdue': '有',
  'number_of_months_overdue_in_the_last_5_years': '11个月',
  'number_of_months_overdue_for_more_than_90_days': '5个月'},
 {'card_issuing_d

#### == 备注

1. 已忽略最后一行 `2014 年 3 月，该机构声明：该客户委托 XX 公司偿还贷款，因开发公司不按时还款导致出现多次逾期。`
   - （处理逻辑是啥，会有别的类似内容吗）
3. !!! `确定关键字段` 和 `手工矫正` 部分需要专业人员进行补充改正
4. 英文翻译需业务人员矫正
5. 操作细节需根据ocr返回结果调整

### 透支超过 60 天的准贷记卡账户明细如下：
略 -- 具体思路同 `信用卡 -- 逾期贷记卡`

#### -- 定义关键字段

- 结果矫正逻辑需人工总结
- 关键字段需人工提前确定：

In [184]:
xyk_tzc60_schema = ['发卡日期', '发卡机构','账户币种','卡片尾号', '销户日期']
xyk_tzc60_schema.extend(['信用额度截至日期','信用额度', '透支余额', '当前透支是否超过60天'])
xyk_tzc60_schema.extend(['近5年透支超过60天月数','透支超过90天月数']) 

{k:key_2_English(k) for k in xyk_tzc60_schema}

{'发卡日期': 'card_issuing_date',
 '发卡机构': 'card_issuer',
 '账户币种': 'account_currency',
 '卡片尾号': 'card_suffix',
 '销户日期': 'account_canceling_date',
 '信用额度截至日期': 'reference_date',
 '信用额度': 'credit_limit',
 '透支余额': 'overdraft_balance',
 '当前透支是否超过60天': 'whether_currently_overdrawn_for_more_than_60_days',
 '近5年透支超过60天月数': 'number_of_months_overdrawn_over_60_days_in_last_5_years',
 '透支超过90天月数': 'number_of_months_overdrawn_over_90_days'}

In [94]:
update_trans_dict("透支超过60天的准贷记卡账户明细如下" ,"overdrawn_quasi_credit_card_accounts_exceeding_60_days")

In [40]:
# update_trans_dict("近5年透支超过60天月数","number_of_months_overdrawn_over_60_days_in_last_5_years")
# update_trans_dict("透支超过90天月数", "number_of_months_overdrawn_over_90_days")

#### -- 数据

In [185]:
xyk_tzc60_data = [
    "2005年6月30日中国银行北京分行发放的准贷记卡（人民币账户，卡片尾号：0999）。截至2016年7月，信用额度10,000，透支余额5,000，当前透支超过60天。最近5年内有6个月透支超过60天，其中3个月透支超过90天。",
    "2006年3月10日上海浦东发展银行北京分行发放的准贷记卡（人民币账户，卡片尾号：0022），2015年12月销户。最近5年内有20个月透支超过60天，其中16个月透支超过90天。"
]

xyk_tzc60_data

['2005年6月30日中国银行北京分行发放的准贷记卡（人民币账户，卡片尾号：0999）。截至2016年7月，信用额度10,000，透支余额5,000，当前透支超过60天。最近5年内有6个月透支超过60天，其中3个月透支超过90天。',
 '2006年3月10日上海浦东发展银行北京分行发放的准贷记卡（人民币账户，卡片尾号：0022），2015年12月销户。最近5年内有20个月透支超过60天，其中16个月透支超过90天。']

#### -- uie base

In [186]:
results = []
for case in xyk_tzc60_data:
    result = extract_info_from_taskflow(case=case, schema=xyk_tzc60_schema)
    results.append(result)

In [55]:
i = 0
print(xyk_tzc60_data[i])
display(pd.DataFrame(list(results[i].items()), columns=['key', 'value']))

2005年6月30日中国银行北京分行发放的准贷记卡（人民币账户，卡片尾号：0999）。截至2016年7月，信用额度10,000，透支余额5,000，当前透支超过60天。最近5年内有6个月透支超过60天，其中3个月透支超过90天。


,key,value
0,发卡日期,2005年6月30日
1,发卡机构,中国银行北京分行
2,账户币种,人民币
3,卡片尾号,0999
4,销户日期,None
5,信用额度截至日期,2016年7月
6,信用额度,"10,000"
7,透支余额,"5,000"
8,当前透支是否超过60天,None
9,近5年透支超过60天月数,6个月


- 问题1：无法判断 “当前透支是否超过60天”

In [54]:
i = 1
print(xyk_tzc60_data[i])
display(pd.DataFrame(list(results[i].items()), columns=['key', 'value']))

2006年3月10日上海浦东发展银行北京分行发放的准贷记卡（人民币账户，卡片尾号：0022），2015年12月销户。最近5年内有20个月透支超过60天，其中16个月透支超过90天。


,key,value
0,发卡日期,2006年3月10日
1,发卡机构,上海浦东发展银行北京分行
2,账户币种,人民币
3,卡片尾号,0022
4,销户日期,2015年12月
5,信用额度截至日期,None
6,信用额度,None
7,透支余额,None
8,当前透支是否超过60天,None
9,近5年透支超过60天月数,20个月


#### -- 修正

In [56]:
# def xyk_yqc60_result_fixing(case, result):
#     """
#     矫正逻辑：
#         1. 如果“当前透支超过60天”存在:
#                 `当前透支是否超过60天	`：`是`
#     """
#     case = case.replace(" ", "")  ## 删空格
    
#     if '当前透支超过60天' in case:
#         result["当前透支是否超过60天"] = '是'

#     return result

In [188]:
results_fixed = []
for case, result in zip(xyk_tzc60_data, results):
    result = text_result_fixing(case, result)
    results_fixed.append(result)

In [189]:
for case, result in zip(xyk_tzc60_data, results):
    print(case)
    display(pd.DataFrame(list(result.items()), columns=['key', 'value']))

2005年6月30日中国银行北京分行发放的准贷记卡（人民币账户，卡片尾号：0999）。截至2016年7月，信用额度10,000，透支余额5,000，当前透支超过60天。最近5年内有6个月透支超过60天，其中3个月透支超过90天。


,key,value
0,发卡日期,2005年6月30日
1,发卡机构,中国银行北京分行
2,账户币种,人民币
3,卡片尾号,0999
4,销户日期,None
5,信用额度截至日期,2016年7月
6,信用额度,"10,000"
7,透支余额,"5,000"
8,当前透支是否超过60天,是
9,近5年透支超过60天月数,6个月


2006年3月10日上海浦东发展银行北京分行发放的准贷记卡（人民币账户，卡片尾号：0022），2015年12月销户。最近5年内有20个月透支超过60天，其中16个月透支超过90天。


,key,value
0,发卡日期,2006年3月10日
1,发卡机构,上海浦东发展银行北京分行
2,账户币种,人民币
3,卡片尾号,0022
4,销户日期,2015年12月
5,信用额度截至日期,None
6,信用额度,None
7,透支余额,None
8,当前透支是否超过60天,None
9,近5年透支超过60天月数,20个月


#### -- key_2_en

In [190]:
results_en = []

for result in results_fixed:
    result_en = {key_2_English(k):v for k,v in result.items()}
    results_en.append(result_en)

results_en

[{'card_issuing_date': '2005年6月30日',
  'card_issuer': '中国银行北京分行',
  'account_currency': '人民币',
  'card_suffix': '0999',
  'account_canceling_date': None,
  'reference_date': '2016年7月',
  'credit_limit': '10,000',
  'overdraft_balance': '5,000',
  'whether_currently_overdrawn_for_more_than_60_days': '是',
  'number_of_months_overdrawn_over_60_days_in_last_5_years': '6个月',
  'number_of_months_overdrawn_over_90_days': '3个月'},
 {'card_issuing_date': '2006年3月10日',
  'card_issuer': '上海浦东发展银行北京分行',
  'account_currency': '人民币',
  'card_suffix': '0022',
  'account_canceling_date': '2015年12月',
  'reference_date': None,
  'credit_limit': None,
  'overdraft_balance': None,
  'whether_currently_overdrawn_for_more_than_60_days': None,
  'number_of_months_overdrawn_over_60_days_in_last_5_years': '20个月',
  'number_of_months_overdrawn_over_90_days': '16个月'}]

### 从未逾期过的贷记卡及透支未超过 60 天的准贷记卡账户明细如下：
略 -- 具体思路同 `信用卡 -- 逾期贷记卡`

#### -- 定义关键字段

In [95]:
en = 'non_overdue_credit_cards_and_quasi_credit_card_accounts_with_overdrafts_less_than_60_day'
ch = '从未逾期过的贷记卡及透支未超过60天的准贷记卡账户明细如下'

update_trans_dict(ch, en)

- 结果矫正逻辑需人工总结
- 关键字段需人工提前确定：

In [191]:
xyk_wyqwtz_schema = ['发卡日期', '发卡机构','账户币种','卡片尾号', '销户日期']
xyk_wyqwtz_schema.extend(['信用额度截至日期', '是否激活', '信用额度', '已使用额度', '总余额', '未出单的大额专项分期余额'])

{k:key_2_English(k) for k in xyk_wyqwtz_schema}

{'发卡日期': 'card_issuing_date',
 '发卡机构': 'card_issuer',
 '账户币种': 'account_currency',
 '卡片尾号': 'card_suffix',
 '销户日期': 'account_canceling_date',
 '信用额度截至日期': 'reference_date',
 '是否激活': 'is_active',
 '信用额度': 'credit_limit',
 '已使用额度': 'credits_used',
 '总余额': 'total_balance',
 '未出单的大额专项分期余额': 'large_amount_of_unissued_special_installment_balance'}

In [93]:
xyk_wyqwtz_schema

['发卡日期',
 '发卡机构',
 '账户币种',
 '卡片尾号',
 '销户日期',
 '信用额度截至日期',
 '是否激活',
 '信用额度',
 '已使用额度',
 '总余额',
 '未出单的大额专项分期余额']

#### -- 数据

In [192]:
xyk_wyqwtz_data = [
    "2005年6月30日中国光大银行北京分行发放的贷记卡（美元账户，卡片尾号：0222）。截至2016年7月，信用额度6,800，余额5,000（含未出单的大额专项分期余额4,000）。",
    "2005年7月30日中国光大银行南京分行发放的贷记卡（美元账户）。截至2016年5月，信用额度6,800，已使用额度5,000。",
    "2006年7月1日招商银行发放的准贷记卡（人民币账户，卡片尾号：1111），2009年12月销户。",
    "2007年6月30日中国光大银行北京分行发放的贷记卡（人民币账户，卡片尾号：2122）。截至2016年7月尚未激活。"
]

xyk_wyqwtz_data

['2005年6月30日中国光大银行北京分行发放的贷记卡（美元账户，卡片尾号：0222）。截至2016年7月，信用额度6,800，余额5,000（含未出单的大额专项分期余额4,000）。',
 '2005年7月30日中国光大银行南京分行发放的贷记卡（美元账户）。截至2016年5月，信用额度6,800，已使用额度5,000。',
 '2006年7月1日招商银行发放的准贷记卡（人民币账户，卡片尾号：1111），2009年12月销户。',
 '2007年6月30日中国光大银行北京分行发放的贷记卡（人民币账户，卡片尾号：2122）。截至2016年7月尚未激活。']

#### -- uie-base

In [193]:
results = []
for case in xyk_wyqwtz_data:
    result = extract_info_from_taskflow(case=case, schema=xyk_wyqwtz_schema)
    results.append(result)

In [80]:
i = 0
print(xyk_wyqwtz_data[i])
display(pd.DataFrame(list(results[i].items()), columns=['key', 'value']))

2005年6月30日中国光大银行北京分行发放的贷记卡（美元账户，卡片尾号：0222）。截至2016年7月，信用额度6,800，余额5,000（含未出单的大额专项分期余额4,000）。


,key,value
0,发卡日期,2005年6月30日
1,发卡机构,中国光大银行北京分行
2,账户币种,美元
3,卡片尾号,0222
4,销户日期,None
5,信用额度截至日期,2016年7月
6,是否激活,None
7,信用额度,"6,800"
8,已使用额度,"6,800"
9,总余额,"5,000"


- 问题1：`已使用额度`不存在，但识别成`信用额度`

In [82]:
i = 1
print(xyk_wyqwtz_data[i])
display(pd.DataFrame(list(results[i].items()), columns=['key', 'value']))

2005年7月30日中国光大银行南京分行发放的贷记卡（美元账户）。截至2016年5月，信用额度6,800，已使用额度5,000。


,key,value
0,发卡日期,2005年7月30日
1,发卡机构,中国光大银行南京分行
2,账户币种,美元
3,卡片尾号,None
4,销户日期,None
5,信用额度截至日期,2016年5月
6,是否激活,None
7,信用额度,"6,800"
8,已使用额度,"5,000"
9,总余额,None


In [83]:
i = 2
print(xyk_wyqwtz_data[i])
display(pd.DataFrame(list(results[i].items()), columns=['key', 'value']))

2006年7月1日招商银行发放的准贷记卡（人民币账户，卡片尾号：1111），2009年12月销户。


,key,value
0,发卡日期,2006年7月1日
1,发卡机构,招商银行
2,账户币种,人民币
3,卡片尾号,1111
4,销户日期,2009年12月
5,信用额度截至日期,None
6,是否激活,None
7,信用额度,None
8,已使用额度,None
9,总余额,None


In [84]:
i = 3
print(xyk_wyqwtz_data[i])
display(pd.DataFrame(list(results[i].items()), columns=['key', 'value']))

2007年6月30日中国光大银行北京分行发放的贷记卡（人民币账户，卡片尾号：2122）。截至2016年7月尚未激活。


,key,value
0,发卡日期,2007年6月30日
1,发卡机构,中国光大银行北京分行
2,账户币种,人民币
3,卡片尾号,2122
4,销户日期,None
5,信用额度截至日期,2016年7月
6,是否激活,None
7,信用额度,None
8,已使用额度,None
9,总余额,None


- 问题2：`尚未激活`无法识别

#### -- 修正

In [89]:
# def xyk_wyqwtz_result_fixing(case, result):
#     """
#     修正逻辑：
#         1. 若原文不含有`已使用额度`:
#                 字段`已使用额度`改为None
#         2. 若原文含有 `尚未激活`：
#                 字段 `是否激活`改为是
#     """
#     case = case.replace(" ", "")  ## 删空格
    
#     if '已使用额度' not in case:
#         result["已使用额度"] = None

#     if '尚未激活' in case:
#         result["是否激活"] = '否'

#     return result

In [195]:
results_fixed = []
for case, result in zip(xyk_wyqwtz_data, results):
    result = text_result_fixing(case, result)
    results_fixed.append(result)

In [196]:
for case, result in zip(xyk_wyqwtz_data, results):
    print(case)
    display(pd.DataFrame(list(result.items()), columns=['key', 'value']))

2005年6月30日中国光大银行北京分行发放的贷记卡（美元账户，卡片尾号：0222）。截至2016年7月，信用额度6,800，余额5,000（含未出单的大额专项分期余额4,000）。


,key,value
0,发卡日期,2005年6月30日
1,发卡机构,中国光大银行北京分行
2,账户币种,美元
3,卡片尾号,0222
4,销户日期,None
5,信用额度截至日期,2016年7月
6,是否激活,None
7,信用额度,"6,800"
8,已使用额度,None
9,总余额,"5,000"


2005年7月30日中国光大银行南京分行发放的贷记卡（美元账户）。截至2016年5月，信用额度6,800，已使用额度5,000。


,key,value
0,发卡日期,2005年7月30日
1,发卡机构,中国光大银行南京分行
2,账户币种,美元
3,卡片尾号,None
4,销户日期,None
5,信用额度截至日期,2016年5月
6,是否激活,None
7,信用额度,"6,800"
8,已使用额度,"5,000"
9,总余额,None


2006年7月1日招商银行发放的准贷记卡（人民币账户，卡片尾号：1111），2009年12月销户。


,key,value
0,发卡日期,2006年7月1日
1,发卡机构,招商银行
2,账户币种,人民币
3,卡片尾号,1111
4,销户日期,2009年12月
5,信用额度截至日期,None
6,是否激活,None
7,信用额度,None
8,已使用额度,None
9,总余额,None


2007年6月30日中国光大银行北京分行发放的贷记卡（人民币账户，卡片尾号：2122）。截至2016年7月尚未激活。


,key,value
0,发卡日期,2007年6月30日
1,发卡机构,中国光大银行北京分行
2,账户币种,人民币
3,卡片尾号,2122
4,销户日期,None
5,信用额度截至日期,2016年7月
6,是否激活,否
7,信用额度,None
8,已使用额度,None
9,总余额,None


#### -- key_2_en

In [197]:
results_en = []

for result in results_fixed:
    result_en = {key_2_English(k):v for k,v in result.items()}
    results_en.append(result_en)

results_en

[{'card_issuing_date': '2005年6月30日',
  'card_issuer': '中国光大银行北京分行',
  'account_currency': '美元',
  'card_suffix': '0222',
  'account_canceling_date': None,
  'reference_date': '2016年7月',
  'is_active': None,
  'credit_limit': '6,800',
  'credits_used': None,
  'total_balance': '5,000',
  'large_amount_of_unissued_special_installment_balance': '4,000'},
 {'card_issuing_date': '2005年7月30日',
  'card_issuer': '中国光大银行南京分行',
  'account_currency': '美元',
  'card_suffix': None,
  'account_canceling_date': None,
  'reference_date': '2016年5月',
  'is_active': None,
  'credit_limit': '6,800',
  'credits_used': '5,000',
  'total_balance': None,
  'large_amount_of_unissued_special_installment_balance': None},
 {'card_issuing_date': '2006年7月1日',
  'card_issuer': '招商银行',
  'account_currency': '人民币',
  'card_suffix': '1111',
  'account_canceling_date': '2009年12月',
  'reference_date': None,
  'is_active': None,
  'credit_limit': None,
  'credits_used': None,
  'total_balance': None,
  'large_amount_of_uni

## 贷款

### 发生过逾期的账户明细如下：
略 -- 具体思路同 `信用卡 -- 逾期贷记卡`

#### -- 定义字段

- 结果矫正逻辑需人工总结
- 关键字段需人工提前确定：

In [83]:
print(key_2_English("信贷记录"), key_2_English("贷款"), key_2_English("发生过逾期的账户明细如下"))

credit_records loans account_details_with_overdue_occurrences


In [16]:
dk_yq_schema = ['贷款发放日期', '贷款发放机构', '贷款发放数额', '贷款币种','贷款种类', '是否为其他机构转入', '贷款到期日期']
dk_yq_schema.extend(['余额截至日期', '余额数目', '当前有无逾期'])
dk_yq_schema.extend(['近5年处于逾期状态月数','逾期超过90天月数'])

{k:key_2_English(k) for k in dk_yq_schema}

{'贷款发放日期': 'loan_release_date',
 '贷款发放机构': 'loan_issuer',
 '贷款发放数额': 'loan_disbursement_amount',
 '贷款币种': 'loan_currency',
 '贷款种类': 'loan_type',
 '是否为其他机构转入': 'whether_to_transfer_to_another_institution',
 '贷款到期日期': 'loan_maturity_date',
 '余额截至日期': 'balance_reference_date',
 '余额数目': 'remaining_balance',
 '当前有无逾期': 'is_currently_overdue',
 '近5年处于逾期状态月数': 'number_of_months_overdue_in_the_last_5_years',
 '逾期超过90天月数': 'number_of_months_overdue_for_more_than_90_days'}

#### -- data

In [5]:
dk_yq_data = [
    "2008年8月30日中国农业银行北京发放的600,000元（美元折人民币）住房贷款（其他机构转入），2028年8月30日到期。截至2016年7月，余额为572,750，当前有逾期。最近5年内有1个月处于逾期状态，没有发生过90天以上逾期。",
    "2008年9月30日中国农业银行北京分行发放的100,000元（人民币）汽车贷款，2018年8月30日到期。截至2016年7月，余额为72,750，当前无逾期。最近5年内有2个月处于逾期状态，没有发生过90天以上逾期。"
]

dk_yq_data

['2008年8月30日中国农业银行北京发放的600,000元（美元折人民币）住房贷款（其他机构转入），2028年8月30日到期。截至2016年7月，余额为572,750，当前有逾期。最近5年内有1个月处于逾期状态，没有发生过90天以上逾期。',
 '2008年9月30日中国农业银行北京分行发放的100,000元（人民币）汽车贷款，2018年8月30日到期。截至2016年7月，余额为72,750，当前无逾期。最近5年内有2个月处于逾期状态，没有发生过90天以上逾期。']

#### -- uie base

In [45]:
results = []
for case in dk_yq_data:
    result = extract_info_from_taskflow(case=case, schema=dk_yq_schema)
    results.append(result)

In [46]:
i = 0
print(dk_yq_data[i])
display(pd.DataFrame(list(results[i].items()), columns=['key', 'value']))

2008年8月30日中国农业银行北京发放的600,000元（美元折人民币）住房贷款（其他机构转入），2028年8月30日到期。截至2016年7月，余额为572,750，当前有逾期。最近5年内有1个月处于逾期状态，没有发生过90天以上逾期。


,key,value
0,贷款发放日期,2008年8月30日
1,贷款发放机构,中国农业银行
2,贷款发放数额,"600,000元"
3,贷款币种,None
4,贷款种类,住房贷款
5,是否为其他机构转入,None
6,贷款到期日期,2028年8月30日
7,余额截至日期,2016年7月
8,余额数目,"572,750"
9,当前有无逾期,None


- ！贷款币种
- ！是否为其他机构转入
- ！当前有无逾期

In [47]:
i = 1
print(dk_yq_data[i])
display(pd.DataFrame(list(results[i].items()), columns=['key', 'value']))

2008年9月30日中国农业银行北京分行发放的100,000元（人民币）汽车贷款，2018年8月30日到期。截至2016年7月，余额为72,750，当前无逾期。最近5年内有2个月处于逾期状态，没有发生过90天以上逾期。


,key,value
0,贷款发放日期,2008年9月30日
1,贷款发放机构,中国农业银行北京分行
2,贷款发放数额,"100,000元"
3,贷款币种,None
4,贷款种类,None
5,是否为其他机构转入,None
6,贷款到期日期,2018年8月30日
7,余额截至日期,2016年7月
8,余额数目,"72,750"
9,当前有无逾期,None


In [ ]:
- 当前有无逾期
- 贷款种类

In [50]:
case = "2008年9月30日中国农业银行北京分行发放的100,000元（人民币）汽车贷款。当前无逾期。最近5年内有2个月处于逾期状态，其中14个月逾期超过90天。"
extract_info_from_taskflow(case=case, schema=dk_yq_schema)

{'贷款发放日期': '2008年9月30日',
 '贷款发放机构': '中国农业银行北京分行',
 '贷款发放数额': '100,000元',
 '贷款币种': None,
 '贷款种类': None,
 '是否为其他机构转入': None,
 '贷款到期日期': '2008年9月30日',
 '余额截至日期': '2008年9月30日',
 '余额数目': '100,000元',
 '当前有无逾期': None,
 '近5年处于逾期状态月数': '2个月',
 '逾期超过90天月数': '14个月'}

- ！截至
- ！到期

#### -- 修正

In [77]:
results_fixed = []
for case, result in zip(dk_yq_data, results):
    result = text_result_fixing(case, result)
    results_fixed.append(result)

In [79]:
for case, result in zip(dk_yq_data, results_fixed):
    print(case)
    display(pd.DataFrame(list(result.items()), columns=['key', 'value']))

2008年8月30日中国农业银行北京发放的600,000元（美元折人民币）住房贷款（其他机构转入），2028年8月30日到期。截至2016年7月，余额为572,750，当前有逾期。最近5年内有1个月处于逾期状态，没有发生过90天以上逾期。


,key,value
0,贷款发放日期,2008年8月30日
1,贷款发放机构,中国农业银行
2,贷款发放数额,"600,000元"
3,贷款币种,美元折人民币
4,贷款种类,住房贷款
5,是否为其他机构转入,是
6,贷款到期日期,2028年8月30日
7,余额截至日期,2016年7月
8,余额数目,"572,750"
9,当前有无逾期,有


2008年9月30日中国农业银行北京分行发放的100,000元（人民币）汽车贷款，2018年8月30日到期。截至2016年7月，余额为72,750，当前无逾期。最近5年内有2个月处于逾期状态，没有发生过90天以上逾期。


,key,value
0,贷款发放日期,2008年9月30日
1,贷款发放机构,中国农业银行北京分行
2,贷款发放数额,"100,000元"
3,贷款币种,人民币
4,贷款种类,汽车贷款
5,是否为其他机构转入,None
6,贷款到期日期,2018年8月30日
7,余额截至日期,2016年7月
8,余额数目,"72,750"
9,当前有无逾期,无


#### -- key 2 en

In [80]:
results_en = []

for result in results_fixed:
    result_en = {key_2_English(k):v for k,v in result.items()}
    results_en.append(result_en)

results_en

[{'loan_release_date': '2008年8月30日',
  'loan_issuer': '中国农业银行',
  'loan_disbursement_amount': '600,000元',
  'loan_currency': '美元折人民币',
  'loan_type': '住房贷款',
  'whether_to_transfer_to_another_institution': '是',
  'loan_maturity_date': '2028年8月30日',
  'balance_reference_date': '2016年7月',
  'remaining_balance': '572,750',
  'is_currently_overdue': '有',
  'number_of_months_overdue_in_the_last_5_years': '1个月',
  'number_of_months_overdue_for_more_than_90_days': None},
 {'loan_release_date': '2008年9月30日',
  'loan_issuer': '中国农业银行北京分行',
  'loan_disbursement_amount': '100,000元',
  'loan_currency': '人民币',
  'loan_type': '汽车贷款',
  'whether_to_transfer_to_another_institution': None,
  'loan_maturity_date': '2018年8月30日',
  'balance_reference_date': '2016年7月',
  'remaining_balance': '72,750',
  'is_currently_overdue': '无',
  'number_of_months_overdue_in_the_last_5_years': '2个月',
  'number_of_months_overdue_for_more_than_90_days': None}]

### 从未逾期过的账户明细如下：
略 -- 具体思路同 `信用卡 -- 逾期贷记卡`

#### -- keys

- 结果矫正逻辑需人工总结
- 关键字段需人工提前确定：

In [126]:
key_2_English("从未逾期过的账户明细如下")

'account_details_with_no_overdue_occurrences'

In [15]:
dk_wyq_schema = ['贷款发放日期', '贷款发放机构', '贷款发放数额', '贷款币种','贷款种类', '结清日期', '贷款到期日期']
dk_wyq_schema.extend(['余额截至日期', '余额数目'])

{k:key_2_English(k) for k in dk_wyq_schema}

{'贷款发放日期': 'loan_release_date',
 '贷款发放机构': 'loan_issuer',
 '贷款发放数额': 'loan_disbursement_amount',
 '贷款币种': 'loan_currency',
 '贷款种类': 'loan_type',
 '结清日期': 'date_of_settlement',
 '贷款到期日期': 'loan_maturity_date',
 '余额截至日期': 'balance_reference_date',
 '余额数目': 'remaining_balance'}

In [16]:
dk_wyq_schema

['贷款发放日期',
 '贷款发放机构',
 '贷款发放数额',
 '贷款币种',
 '贷款种类',
 '结清日期',
 '贷款到期日期',
 '余额截至日期',
 '余额数目']

#### -- data

In [101]:
# raw = """
# 3. 2010 年 5 月 8 日北京银行金融街支行发放的 200,000 元（人民币）住房贷款，2029 年 5 月 8 日到期。截至 2016 年 7 月，余
# 额 50,000。
# 4. 2012 年 5 月 8 日中信银行知春路支行发放的 100,000 元（人民币）个人经营性贷款，2019 年 5 月 8 日到期。截至 2016 年 7
# 月，余额 50,000。
# 5. 2013 年 4 月 15 日福特汽车金融公司发放的 100,000 元（人民币）汽车贷款，2013 年 4 月 15 日到期。截至 2016 年 7 月，余
# 额 50,000。
# 6. 2011 年 9 月 10 日招商银行金融街支行发放的 300,000 元（人民币）住房贷款，2015 年 12 月已结清。
# """

In [102]:
# cases = split_cases_by_idx_from_paragraph(raw)
# cases

['2010年5月8日北京银行金融街支行发放的200,000元（人民币）住房贷款，2029年5月8日到期。截至2016年7月，余额50,000。',
 '2012年5月8日中信银行知春路支行发放的100,000元（人民币）个人经营性贷款，2019年5月8日到期。截至2016年7月，余额50,000。',
 '2013年4月15日福特汽车金融公司发放的100,000元（人民币）汽车贷款，2013年4月15日到期。截至2016年7月，余额50,000。',
 '2011年9月10日招商银行金融街支行发放的300,000元（人民币）住房贷款，2015年12月已结清。']

In [130]:
dk_wyq_data = ['2010年5月8日北京银行金融街支行发放的200,000元（人民币）住房贷款，2029年5月8日到期。截至2016年7月，余额50,000。',
 '2012年5月8日中信银行知春路支行发放的100,000元（人民币）个人经营性贷款，2019年5月8日到期。截至2016年7月，余额50,000。',
 '2013年4月15日福特汽车金融公司发放的100,000元（人民币）汽车贷款，2013年4月15日到期。截至2016年7月，余额50,000。',
 '2011年9月10日招商银行金融街支行发放的300,000元（人民币）住房贷款，2015年12月已结清。']

dk_wyq_data

['2010年5月8日北京银行金融街支行发放的200,000元（人民币）住房贷款，2029年5月8日到期。截至2016年7月，余额50,000。',
 '2012年5月8日中信银行知春路支行发放的100,000元（人民币）个人经营性贷款，2019年5月8日到期。截至2016年7月，余额50,000。',
 '2013年4月15日福特汽车金融公司发放的100,000元（人民币）汽车贷款，2013年4月15日到期。截至2016年7月，余额50,000。',
 '2011年9月10日招商银行金融街支行发放的300,000元（人民币）住房贷款，2015年12月已结清。']

#### -- uie base

In [114]:
dk_wyq_results = []
for case in dk_wyq_data:
    result = extract_info_from_taskflow(case=case, schema=dk_wyq_schema)
    dk_wyq_results.append(result)

In [115]:
i = 0
print(dk_wyq_data[i])
display(pd.DataFrame(list(dk_wyq_results[i].items()), columns=['key', 'value']))

2010年5月8日北京银行金融街支行发放的200,000元（人民币）住房贷款，2029年5月8日到期。截至2016年7月，余额50,000。


,key,value
0,贷款发放日期,2010年5月8日
1,贷款发放机构,北京银行金融街支行
2,贷款发放数额,"200,000元"
3,贷款币种,None
4,贷款种类,None
5,结清日期,2029年5月8日
6,贷款到期日期,2029年5月8日
7,余额截至日期,2016年7月
8,余额数目,"50,000"


- ！贷款币种
- ！贷款种类
- ！结清日期

In [116]:
i = 1
print(dk_wyq_data[i])
display(pd.DataFrame(list(dk_wyq_results[i].items()), columns=['key', 'value']))

2012年5月8日中信银行知春路支行发放的100,000元（人民币）个人经营性贷款，2019年5月8日到期。截至2016年7月，余额50,000。


,key,value
0,贷款发放日期,2012年5月8日
1,贷款发放机构,中信银行知春路支行
2,贷款发放数额,"100,000元"
3,贷款币种,None
4,贷款种类,None
5,结清日期,2019年5月8日
6,贷款到期日期,2019年5月8日
7,余额截至日期,2016年7月
8,余额数目,"50,000"


- 贷款币种，贷款种类，结清日期

In [117]:
i = 2
print(dk_wyq_data[i])
display(pd.DataFrame(list(dk_wyq_results[i].items()), columns=['key', 'value']))

2013年4月15日福特汽车金融公司发放的100,000元（人民币）汽车贷款，2013年4月15日到期。截至2016年7月，余额50,000。


,key,value
0,贷款发放日期,2013年4月15日
1,贷款发放机构,福特汽车金融公司
2,贷款发放数额,"100,000元"
3,贷款币种,None
4,贷款种类,None
5,结清日期,2013年4月15日
6,贷款到期日期,2013年4月15日
7,余额截至日期,2016年7月
8,余额数目,"50,000"


- 贷款币种，贷款种类，结清日期

In [118]:
i = 3
print(dk_wyq_data[i])
display(pd.DataFrame(list(dk_wyq_results[i].items()), columns=['key', 'value']))

2011年9月10日招商银行金融街支行发放的300,000元（人民币）住房贷款，2015年12月已结清。


,key,value
0,贷款发放日期,2011年9月10日
1,贷款发放机构,招商银行金融街支行
2,贷款发放数额,"300,000元"
3,贷款币种,None
4,贷款种类,None
5,结清日期,2015年12月
6,贷款到期日期,2011年9月10日
7,余额截至日期,2015年12月
8,余额数目,"300,000元"


- ！贷款到期日期
- ！余额截至日期
- ！余额数目
- 贷款币种 + 贷款种类 + 结清日期


#### -- fix

In [121]:
dk_wyq_results_fixed = []
for case, result in zip(dk_wyq_data, dk_wyq_results):
    result = text_result_fixing(case, result)
    dk_wyq_results_fixed.append(result)

In [123]:
for case, result in zip(dk_wyq_data, dk_wyq_results_fixed):
    print(case)
    display(pd.DataFrame(list(result.items()), columns=['key', 'value']))

2010年5月8日北京银行金融街支行发放的200,000元（人民币）住房贷款，2029年5月8日到期。截至2016年7月，余额50,000。


,key,value
0,贷款发放日期,2010年5月8日
1,贷款发放机构,北京银行金融街支行
2,贷款发放数额,"200,000元"
3,贷款币种,人民币
4,贷款种类,住房贷款
5,结清日期,None
6,贷款到期日期,2029年5月8日
7,余额截至日期,2016年7月
8,余额数目,"50,000"


2012年5月8日中信银行知春路支行发放的100,000元（人民币）个人经营性贷款，2019年5月8日到期。截至2016年7月，余额50,000。


,key,value
0,贷款发放日期,2012年5月8日
1,贷款发放机构,中信银行知春路支行
2,贷款发放数额,"100,000元"
3,贷款币种,人民币
4,贷款种类,个人经营性贷款
5,结清日期,None
6,贷款到期日期,2019年5月8日
7,余额截至日期,2016年7月
8,余额数目,"50,000"


2013年4月15日福特汽车金融公司发放的100,000元（人民币）汽车贷款，2013年4月15日到期。截至2016年7月，余额50,000。


,key,value
0,贷款发放日期,2013年4月15日
1,贷款发放机构,福特汽车金融公司
2,贷款发放数额,"100,000元"
3,贷款币种,人民币
4,贷款种类,汽车贷款
5,结清日期,None
6,贷款到期日期,2013年4月15日
7,余额截至日期,2016年7月
8,余额数目,"50,000"


2011年9月10日招商银行金融街支行发放的300,000元（人民币）住房贷款，2015年12月已结清。


,key,value
0,贷款发放日期,2011年9月10日
1,贷款发放机构,招商银行金融街支行
2,贷款发放数额,"300,000元"
3,贷款币种,人民币
4,贷款种类,住房贷款
5,结清日期,2015年12月
6,贷款到期日期,None
7,余额截至日期,None
8,余额数目,None


#### -- en

In [124]:
dk_wyq_results_en = []

for result in dk_wyq_results_fixed:
    result_en = {key_2_English(k):v for k,v in result.items()}
    dk_wyq_results_en.append(result_en)

dk_wyq_results_en

[{'loan_release_date': '2010年5月8日',
  'loan_issuer': '北京银行金融街支行',
  'loan_disbursement_amount': '200,000元',
  'loan_currency': '人民币',
  'loan_type': '住房贷款',
  'date_of_settlement': None,
  'loan_maturity_date': '2029年5月8日',
  'balance_reference_date': '2016年7月',
  'remaining_balance': '50,000'},
 {'loan_release_date': '2012年5月8日',
  'loan_issuer': '中信银行知春路支行',
  'loan_disbursement_amount': '100,000元',
  'loan_currency': '人民币',
  'loan_type': '个人经营性贷款',
  'date_of_settlement': None,
  'loan_maturity_date': '2019年5月8日',
  'balance_reference_date': '2016年7月',
  'remaining_balance': '50,000'},
 {'loan_release_date': '2013年4月15日',
  'loan_issuer': '福特汽车金融公司',
  'loan_disbursement_amount': '100,000元',
  'loan_currency': '人民币',
  'loan_type': '汽车贷款',
  'date_of_settlement': None,
  'loan_maturity_date': '2013年4月15日',
  'balance_reference_date': '2016年7月',
  'remaining_balance': '50,000'},
 {'loan_release_date': '2011年9月10日',
  'loan_issuer': '招商银行金融街支行',
  'loan_disbursement_amount': '300,000

## 其他业务

### 发生过逾期的账户明细如下：
简写 -- 具体思路同 `信用卡 -- 逾期贷记卡`

#### -- 定义关键字段

In [80]:
update_trans_dict("业务到期日期", "business_end_date")

In [81]:
qtyw_fsgyq_schema = ["业务种类", "业务开始日期", "业务办理单位", "业务办理金额", "业务货币种类", "业务到期日期"]
qtyw_fsgyq_schema.extend(["截至日期","是否为呆账", "是否变成司法追偿", "最近还款日期", "余额数目"])
qtyw_fsgyq_schema.extend(['当前有无逾期', '近5年处于逾期状态月数','逾期超过90天月数'])

{k:key_2_English(k) for k in qtyw_fsgyq_schema}

{'业务种类': 'business_type',
 '业务开始日期': 'start_date_of_business',
 '业务办理单位': 'business_handling_unit',
 '业务办理金额': 'business_transaction_amount',
 '业务货币种类': 'business_currency_type',
 '业务到期日期': 'business_end_date',
 '截至日期': 'reference_date',
 '是否为呆账': 'whether_it_is_a_bad_debt',
 '是否变成司法追偿': 'whether_it_becomes_a_judicial_recourse',
 '最近还款日期': 'last_repayment_date',
 '余额数目': 'remaining_balance',
 '当前有无逾期': 'is_currently_overdue',
 '近5年处于逾期状态月数': 'number_of_months_overdue_in_the_last_5_years',
 '逾期超过90天月数': 'number_of_months_overdue_for_more_than_90_days'}

#### -- 预处理数据

In [127]:
# ## 预期ocr识别结果
# qtyw_fsgyq_data = """
# \n1. 2005年10月20日建元资本（中国）融资租赁有限公司办理的500,000元（人民币）融资租赁业务，2020年10月20日到期。\n截至2011年11月，已变成呆账，最近一次还款日期为2010年11月25日，余额150,000。\n2. 2005年9月22日建元资本（中国）融资租赁有限公司办理的500,000元（人民币）融资租赁业务，2020年9月22日到期。\n截至2016年7月，余额400,000，当前无逾期。最近5年内有3个月处于逾期状态，没有发生过90天以上逾期。\n3. 2006年10月20日中信证券公司办理的500,000元（人民币）约定购回式证券交易，2007年10月19日到期。截至2008年1\n月，已变成司法追偿，最近一次还款日期为2007年10月15日，余额150,000。\n4. 2009年9月22日海通证券公司办理的500,000元（人民币）约定购回式证券交易，2010年9月21日到期。截至2016年7\n月，余额500,000，当前有逾期。最近5年内有1个月处于逾期状态，没有发生过90天以上逾期。\n
# """
# qtyw_fsgyq_data

'\n\n1. 2005年10月20日建元资本（中国）融资租赁有限公司办理的500,000元（人民币）融资租赁业务，2020年10月20日到期。\n截至2011年11月，已变成呆账，最近一次还款日期为2010年11月25日，余额150,000。\n2. 2005年9月22日建元资本（中国）融资租赁有限公司办理的500,000元（人民币）融资租赁业务，2020年9月22日到期。\n截至2016年7月，余额400,000，当前无逾期。最近5年内有3个月处于逾期状态，没有发生过90天以上逾期。\n3. 2006年10月20日中信证券公司办理的500,000元（人民币）约定购回式证券交易，2007年10月19日到期。截至2008年1\n月，已变成司法追偿，最近一次还款日期为2007年10月15日，余额150,000。\n4. 2009年9月22日海通证券公司办理的500,000元（人民币）约定购回式证券交易，2010年9月21日到期。截至2016年7\n月，余额500,000，当前有逾期。最近5年内有1个月处于逾期状态，没有发生过90天以上逾期。\n\n'

In [128]:
# ## 按条目分段
# qtyw_fsgyq_cases = split_cases_by_idx_from_paragraph(qtyw_fsgyq_data)
# qtyw_fsgyq_cases

['2005年10月20日建元资本（中国）融资租赁有限公司办理的500,000元（人民币）融资租赁业务，2020年10月20日到期。截至2011年11月，已变成呆账，最近一次还款日期为2010年11月25日，余额150,000。',
 '2005年9月22日建元资本（中国）融资租赁有限公司办理的500,000元（人民币）融资租赁业务，2020年9月22日到期。截至2016年7月，余额400,000，当前无逾期。最近5年内有3个月处于逾期状态，没有发生过90天以上逾期。',
 '2006年10月20日中信证券公司办理的500,000元（人民币）约定购回式证券交易，2007年10月19日到期。截至2008年1月，已变成司法追偿，最近一次还款日期为2007年10月15日，余额150,000。',
 '2009年9月22日海通证券公司办理的500,000元（人民币）约定购回式证券交易，2010年9月21日到期。截至2016年7月，余额500,000，当前有逾期。最近5年内有1个月处于逾期状态，没有发生过90天以上逾期。']

In [61]:
qtyw_fsgyq_data = ['2005年10月20日建元资本（中国）融资租赁有限公司办理的500,000元（人民币）融资租赁业务，2020年10月20日到期。截至2011年11月，已变成呆账，最近一次还款日期为2010年11月25日，余额150,000。',
 '2005年9月22日建元资本（中国）融资租赁有限公司办理的500,000元（人民币）融资租赁业务，2020年9月22日到期。截至2016年7月，余额400,000，当前无逾期。最近5年内有3个月处于逾期状态，没有发生过90天以上逾期。',
 '2006年10月20日中信证券公司办理的500,000元（人民币）约定购回式证券交易，2007年10月19日到期。截至2008年1月，已变成司法追偿，最近一次还款日期为2007年10月15日，余额150,000。',
 '2009年9月22日海通证券公司办理的500,000元（人民币）约定购回式证券交易，2010年9月21日到期。截至2016年7月，余额500,000，当前有逾期。最近5年内有1个月处于逾期状态，没有发生过90天以上逾期。']

qtyw_fsgyq_data

['2005年10月20日建元资本（中国）融资租赁有限公司办理的500,000元（人民币）融资租赁业务，2020年10月20日到期。截至2011年11月，已变成呆账，最近一次还款日期为2010年11月25日，余额150,000。',
 '2005年9月22日建元资本（中国）融资租赁有限公司办理的500,000元（人民币）融资租赁业务，2020年9月22日到期。截至2016年7月，余额400,000，当前无逾期。最近5年内有3个月处于逾期状态，没有发生过90天以上逾期。',
 '2006年10月20日中信证券公司办理的500,000元（人民币）约定购回式证券交易，2007年10月19日到期。截至2008年1月，已变成司法追偿，最近一次还款日期为2007年10月15日，余额150,000。',
 '2009年9月22日海通证券公司办理的500,000元（人民币）约定购回式证券交易，2010年9月21日到期。截至2016年7月，余额500,000，当前有逾期。最近5年内有1个月处于逾期状态，没有发生过90天以上逾期。']

#### -- 信息抽取结果

In [135]:
# """ 人工矫正抽取结果 """
# def qtyw_fsgyq_result_fixing(case, result):
#     '''
#     矫正逻辑：
#         1. 若不含有“最近一次还款日期”：
#                 最近还款日期：None
#         2. 若含有 “当前有逾期”：
#                 当前有无逾期：有
#            若含有 “当前无逾期”：
#                 当前有无逾期：无
#         3. 若含有 “已变成呆账”：
#                 是否为呆账: 是
#         4. 业务种类: 
#                 第一个`，`之前 -- 再 -- 最后一个`）`之后
#         5. 若含有“已变成司法追偿”：
#                 是否变成司法追偿：是
#         6. 货币种类：
#                 `办理的` 与 `，` 之间 -- 再 -- `（`与`）`之间
#         7. 业务办理金额：
#                 `办理的`后 + 去除`,` + 数字匹配
#     '''
#     case = case.replace(" ", "")  ## 删空格
    
#     if not '最近一次还款日期' in case:
#         result["最近还款日期"] = None

#     if '当前有逾期' in case:
#         result["当前有无逾期"] = "有"
#     if '当前无逾期' in case:
#         result["当前有无逾期"] = "无"

#     if "已变成呆账" in case:
#         result["是否为呆账"] = "是"
    
#     ywzl_part = case.split("，")
#     if ywzl_part:
#         ywzl_part = ywzl_part[0].split("）")
#         if ywzl_part:
#             ywzl = ywzl_part[-1]
#             result["业务种类"] = ywzl

#     if "已变成司法追偿" in case:
#         result["是否变成司法追偿"] = '是'

#     hbzl = re.search(r'办理的(.*?)，', case)
#     if hbzl:
#         hbzl = re.search(r'\（(.*?)\）', hbzl.group(1))
#         if hbzl:
#             result["货币种类"] = hbzl.group(1)

#     ywblje = case.split("办理的")
#     if ywblje:
#         ywblje = ywblje[1].replace(",","")
#         ywblje = re.search(r'\d+', ywblje)
#         if ywblje:
#             result["业务办理金额"] = ywblje.group(0)
            
#     return result
    

In [82]:
qtyw_fsgyq_results = []
qtyw_fsgyq_results_fixed = []
qtyw_fsgyq_results_fixed_en = []
for case in qtyw_fsgyq_data:
    ## 结果展示
    print("-----------")
    print(case)
    ## 模型提取
    result = extract_info_from_taskflow(case=case, schema=qtyw_fsgyq_schema)
    qtyw_fsgyq_results.append(result)
    ## 结果矫正
    result = text_result_fixing(case, result)
    qtyw_fsgyq_results_fixed.append(result)
    display(pd.DataFrame(list(result.items()), columns=['key', 'value']))
    ## key值转英文
    result = {key_2_English(k):v for k,v in result.items()}
    qtyw_fsgyq_results_fixed_en.append(result)
    

-----------
2005年10月20日建元资本（中国）融资租赁有限公司办理的500,000元（人民币）融资租赁业务，2020年10月20日到期。截至2011年11月，已变成呆账，最近一次还款日期为2010年11月25日，余额150,000。


,key,value
0,业务种类,融资租赁业务
1,业务开始日期,2005年10月20日
2,业务办理单位,建元资本（中国）融资租赁有限公司
3,业务办理金额,500000
4,业务货币种类,人民币
5,业务到期日期,2020年10月20日
6,截至日期,2011年11月
7,是否为呆账,是
8,是否变成司法追偿,None
9,最近还款日期,2010年11月25日


-----------
2005年9月22日建元资本（中国）融资租赁有限公司办理的500,000元（人民币）融资租赁业务，2020年9月22日到期。截至2016年7月，余额400,000，当前无逾期。最近5年内有3个月处于逾期状态，没有发生过90天以上逾期。


,key,value
0,业务种类,融资租赁业务
1,业务开始日期,2005年9月22日
2,业务办理单位,建元资本（中国）融资租赁有限公司
3,业务办理金额,500000
4,业务货币种类,人民币
5,业务到期日期,2020年9月22日
6,截至日期,2016年7月
7,是否为呆账,None
8,是否变成司法追偿,None
9,最近还款日期,None


-----------
2006年10月20日中信证券公司办理的500,000元（人民币）约定购回式证券交易，2007年10月19日到期。截至2008年1月，已变成司法追偿，最近一次还款日期为2007年10月15日，余额150,000。


,key,value
0,业务种类,约定购回式证券交易
1,业务开始日期,2006年10月20日
2,业务办理单位,中信证券公司
3,业务办理金额,500000
4,业务货币种类,人民币
5,业务到期日期,2007年10月19日
6,截至日期,2008年1月
7,是否为呆账,None
8,是否变成司法追偿,是
9,最近还款日期,2007年10月15日


-----------
2009年9月22日海通证券公司办理的500,000元（人民币）约定购回式证券交易，2010年9月21日到期。截至2016年7月，余额500,000，当前有逾期。最近5年内有1个月处于逾期状态，没有发生过90天以上逾期。


,key,value
0,业务种类,约定购回式证券交易
1,业务开始日期,2009年9月22日
2,业务办理单位,海通证券公司
3,业务办理金额,500000
4,业务货币种类,人民币
5,业务到期日期,2010年9月21日
6,截至日期,2016年7月
7,是否为呆账,None
8,是否变成司法追偿,None
9,最近还款日期,None


#### -- en

In [84]:
qtyw_fsgyq_results_fixed_en

[{'business_type': '融资租赁业务',
  'start_date_of_business': '2005年10月20日',
  'business_handling_unit': '建元资本（中国）融资租赁有限公司',
  'business_transaction_amount': '500000',
  'business_currency_type': '人民币',
  'business_end_date': '2020年10月20日',
  'reference_date': '2011年11月',
  'whether_it_is_a_bad_debt': '是',
  'whether_it_becomes_a_judicial_recourse': None,
  'last_repayment_date': '2010年11月25日',
  'remaining_balance': '150,000',
  'is_currently_overdue': None,
  'number_of_months_overdue_in_the_last_5_years': None,
  'number_of_months_overdue_for_more_than_90_days': None},
 {'business_type': '融资租赁业务',
  'start_date_of_business': '2005年9月22日',
  'business_handling_unit': '建元资本（中国）融资租赁有限公司',
  'business_transaction_amount': '500000',
  'business_currency_type': '人民币',
  'business_end_date': '2020年9月22日',
  'reference_date': '2016年7月',
  'whether_it_is_a_bad_debt': None,
  'whether_it_becomes_a_judicial_recourse': None,
  'last_repayment_date': None,
  'remaining_balance': '400,000',
  'is_curr

#### -- 备注

- !! 矫正规则需业务人员完善补充

### 从未逾期过的账户明细如下：
略 -- 具体思路同 `信用卡 -- 逾期贷记卡`

- 结果矫正逻辑需人工总结
- 关键字段需人工提前确定：

#### -- key

In [51]:
qtyw_wyq_schema = ["业务种类", "业务办理日期", "业务办理单位", "业务办理金额", "业务货币种类", "业务到期日期", "结清日期"]
qtyw_wyq_schema.extend(["截至日期", "余额数目"])

{k:key_2_English(k) for k in qtyw_wyq_schema}

{'业务种类': 'business_type',
 '业务办理日期': 'business_handling_date',
 '业务办理单位': 'business_handling_unit',
 '业务办理金额': 'business_transaction_amount',
 '业务货币种类': 'business_currency_type',
 '业务到期日期': 'business_expiration_date',
 '结清日期': 'date_of_settlement',
 '截至日期': 'reference_date',
 '余额数目': 'remaining_balance'}

In [27]:
qtyw_wyq_schema

['业务种类',
 '业务开始日期',
 '业务办理单位',
 '业务办理金额',
 '业务货币种类',
 '业务结束日期',
 '结清日期',
 '截至日期',
 '余额数目']

#### -- data

In [141]:
# raw = """
# 5. 2014 年 9 月 29 日建元资本（中国）融资租赁有限公司办理的 10,000 元（人民币）融资租赁业务，2019 年 12 月 28 日到期。
# 截至 2016 年 6 月，余额 2,000。
# 6. 2015 年 1 月 12 日建元资本（中国）融资租赁有限公司办理的 50,000 元（人民币）融资租赁业务， 2016 年 7 月已结清。
# 第 3 页/共 5 页
# 7. 2015 年 6 月 29 日中信证券公司办理的 80,000 元（人民币）股票质押式回购交易，2018 年 6 月 28 日到期。截至 2016 年 7
# 月，余额 80,000。
# 8. 2015 年 7 月 12 日国泰君安证券公司办理的 50,000 元（人民币）约定购回式证券交易， 2016 年 7 月已结清。
# """

In [142]:
# cases = split_cases_by_idx_from_paragraph(raw)
# cases

['2014年9月29日建元资本（中国）融资租赁有限公司办理的10,000元（人民币）融资租赁业务，2019年12月28日到期。截至2016年6月，余额2,000。',
 '2015年1月12日建元资本（中国）融资租赁有限公司办理的50,000元（人民币）融资租赁业务，2016年7月已结清。第3页/共5页',
 '2015年6月29日中信证券公司办理的80,000元（人民币）股票质押式回购交易，2018年6月28日到期。截至2016年7月，余额80,000。',
 '2015年7月12日国泰君安证券公司办理的50,000元（人民币）约定购回式证券交易，2016年7月已结清。']

In [20]:
qtyw_wyq_data = ['2014年9月29日建元资本（中国）融资租赁有限公司办理的10,000元（人民币）融资租赁业务，2019年12月28日到期。截至2016年6月，余额2,000。',
 '2015年1月12日建元资本（中国）融资租赁有限公司办理的50,000元（人民币）融资租赁业务，2016年7月已结清。',
 '2015年6月29日中信证券公司办理的80,000元（人民币）股票质押式回购交易，2018年6月28日到期。截至2016年7月，余额80,000。',
 '2015年7月12日国泰君安证券公司办理的50,000元（人民币）约定购回式证券交易，2016年7月已结清。']

qtyw_wyq_data

['2014年9月29日建元资本（中国）融资租赁有限公司办理的10,000元（人民币）融资租赁业务，2019年12月28日到期。截至2016年6月，余额2,000。',
 '2015年1月12日建元资本（中国）融资租赁有限公司办理的50,000元（人民币）融资租赁业务，2016年7月已结清。第3页/共5页',
 '2015年6月29日中信证券公司办理的80,000元（人民币）股票质押式回购交易，2018年6月28日到期。截至2016年7月，余额80,000。',
 '2015年7月12日国泰君安证券公司办理的50,000元（人民币）约定购回式证券交易，2016年7月已结清。']

#### -- uie-base

In [52]:
qtyw_wyq_results = []
for case in qtyw_wyq_data:
    result = extract_info_from_taskflow(case=case, schema=qtyw_wyq_schema)
    qtyw_wyq_results.append(result)

In [53]:
i = 0
print(qtyw_wyq_data[i])
display(pd.DataFrame(list(qtyw_wyq_results[i].items()), columns=['key', 'value']))

2014年9月29日建元资本（中国）融资租赁有限公司办理的10,000元（人民币）融资租赁业务，2019年12月28日到期。截至2016年6月，余额2,000。


,key,value
0,业务种类,融资租赁业务
1,业务办理日期,2014年9月29日
2,业务办理单位,建元资本（中国）融资租赁有限公司
3,业务办理金额,"10,000元"
4,业务货币种类,None
5,业务到期日期,2019年12月28日
6,结清日期,2016年6月
7,截至日期,2016年6月
8,余额数目,"2,000"


- ！结清日期
- ！业务货币种类

In [54]:
i = 1
print(qtyw_wyq_data[i])
display(pd.DataFrame(list(qtyw_wyq_results[i].items()), columns=['key', 'value']))

2015年1月12日建元资本（中国）融资租赁有限公司办理的50,000元（人民币）融资租赁业务，2016年7月已结清。第3页/共5页


,key,value
0,业务种类,None
1,业务办理日期,2015年1月12日
2,业务办理单位,建元资本（中国）融资租赁有限公司
3,业务办理金额,"50,000元"
4,业务货币种类,None
5,业务到期日期,2015年1月12日
6,结清日期,2016年7月
7,截至日期,2015年1月12日
8,余额数目,None


- ！业务种类
- 货币种类

In [55]:
i = 2
print(qtyw_wyq_data[i])
display(pd.DataFrame(list(qtyw_wyq_results[i].items()), columns=['key', 'value']))

2015年6月29日中信证券公司办理的80,000元（人民币）股票质押式回购交易，2018年6月28日到期。截至2016年7月，余额80,000。


,key,value
0,业务种类,None
1,业务办理日期,2015年6月29日
2,业务办理单位,中信证券公司
3,业务办理金额,None
4,业务货币种类,None
5,业务到期日期,2018年6月28日
6,结清日期,2018年6月28日
7,截至日期,2016年7月
8,余额数目,"80,000"


- 业务种类
- ！业务办理金额

In [56]:
i = 3
print(qtyw_wyq_data[i])
display(pd.DataFrame(list(qtyw_wyq_results[i].items()), columns=['key', 'value']))

2015年7月12日国泰君安证券公司办理的50,000元（人民币）约定购回式证券交易，2016年7月已结清。


,key,value
0,业务种类,None
1,业务办理日期,2015年7月12日
2,业务办理单位,国泰君安证券公司
3,业务办理金额,None
4,业务货币种类,None
5,业务到期日期,2015年7月12日
6,结清日期,2016年7月
7,截至日期,2015年7月12日
8,余额数目,None


- 业务种类
- 业务办理金额
- 业务货币种类
- 业务到期日期
- 截至日期

#### -- fix

In [66]:
qtyw_wyq_results_fixed = []
for case, result in zip(qtyw_wyq_data, qtyw_wyq_results):
    result = text_result_fixing(case, result)
    qtyw_wyq_results_fixed.append(result)

In [67]:
for case, result in zip(qtyw_wyq_data, qtyw_wyq_results_fixed):
    print(case)
    display(pd.DataFrame(list(result.items()), columns=['key', 'value']))

2014年9月29日建元资本（中国）融资租赁有限公司办理的10,000元（人民币）融资租赁业务，2019年12月28日到期。截至2016年6月，余额2,000。


,key,value
0,业务种类,融资租赁业务
1,业务办理日期,2014年9月29日
2,业务办理单位,建元资本（中国）融资租赁有限公司
3,业务办理金额,10000
4,业务货币种类,人民币
5,业务到期日期,2019年12月28日
6,结清日期,None
7,截至日期,2016年6月
8,余额数目,"2,000"


2015年1月12日建元资本（中国）融资租赁有限公司办理的50,000元（人民币）融资租赁业务，2016年7月已结清。第3页/共5页


,key,value
0,业务种类,融资租赁业务
1,业务办理日期,2015年1月12日
2,业务办理单位,建元资本（中国）融资租赁有限公司
3,业务办理金额,50000
4,业务货币种类,人民币
5,业务到期日期,None
6,结清日期,2016年7月
7,截至日期,None
8,余额数目,None


2015年6月29日中信证券公司办理的80,000元（人民币）股票质押式回购交易，2018年6月28日到期。截至2016年7月，余额80,000。


,key,value
0,业务种类,股票质押式回购交易
1,业务办理日期,2015年6月29日
2,业务办理单位,中信证券公司
3,业务办理金额,80000
4,业务货币种类,人民币
5,业务到期日期,2018年6月28日
6,结清日期,None
7,截至日期,2016年7月
8,余额数目,"80,000"


2015年7月12日国泰君安证券公司办理的50,000元（人民币）约定购回式证券交易，2016年7月已结清。


,key,value
0,业务种类,约定购回式证券交易
1,业务办理日期,2015年7月12日
2,业务办理单位,国泰君安证券公司
3,业务办理金额,50000
4,业务货币种类,人民币
5,业务到期日期,None
6,结清日期,2016年7月
7,截至日期,None
8,余额数目,None


#### -- en

In [70]:
qtyw_wyq_results_en = []

for result in qtyw_wyq_results_fixed:
    result_en = {key_2_English(k):v for k,v in result.items()}
    qtyw_wyq_results_en.append(result_en)

qtyw_wyq_results_en

[{'business_type': '融资租赁业务',
  'business_handling_date': '2014年9月29日',
  'business_handling_unit': '建元资本（中国）融资租赁有限公司',
  'business_transaction_amount': '10000',
  'business_currency_type': '人民币',
  'business_expiration_date': '2019年12月28日',
  'date_of_settlement': None,
  'reference_date': '2016年6月',
  'remaining_balance': '2,000'},
 {'business_type': '融资租赁业务',
  'business_handling_date': '2015年1月12日',
  'business_handling_unit': '建元资本（中国）融资租赁有限公司',
  'business_transaction_amount': '50000',
  'business_currency_type': '人民币',
  'business_expiration_date': None,
  'date_of_settlement': '2016年7月',
  'reference_date': None,
  'remaining_balance': None},
 {'business_type': '股票质押式回购交易',
  'business_handling_date': '2015年6月29日',
  'business_handling_unit': '中信证券公司',
  'business_transaction_amount': '80000',
  'business_currency_type': '人民币',
  'business_expiration_date': '2018年6月28日',
  'date_of_settlement': None,
  'reference_date': '2016年7月',
  'remaining_balance': '80,000'},
 {'business_ty

## 相关还款责任信息
略 -- 具体思路同 `信用卡 -- 逾期贷记卡`

- 结果矫正逻辑需人工总结
- 关键字段需人工提前确定：

#### -- key

In [139]:
xghkzrxx_schema = ["业务办理日期", "办理方姓名", "办理人证件类型", "办理人证件号码", "办理机构", "承担相关还款责任的业务", "责任人类型", "责任金额", "保证合同编号"]
xghkzrxx_schema.extend(["截至日期", "余额数目", "余额币种"])

{k:key_2_English(k) for k in xghkzrxx_schema}

{'业务办理日期': 'business_handling_date',
 '办理方姓名': 'handler_name',
 '办理人证件类型': 'handler_id_type',
 '办理人证件号码': 'handler_id_number',
 '办理机构': 'handling_institution',
 '承担相关还款责任的业务': 'business_bearing_relevant_repayment_responsibilities',
 '责任人类型': 'responsible_person_type',
 '责任金额': 'amount_liable',
 '保证合同编号': 'guarantee_contract_no',
 '截至日期': 'reference_date',
 '余额数目': 'remaining_balance',
 '余额币种': 'balance_currency'}

In [140]:
xghkzrxx_schema

['业务办理日期',
 '办理方姓名',
 '办理人证件类型',
 '办理人证件号码',
 '办理机构',
 '承担相关还款责任的业务',
 '责任人类型',
 '责任金额',
 '保证合同编号',
 '截至日期',
 '余额数目',
 '余额币种']

#### -- data

In [87]:
# raw = """
# 1.2009 年 3 月 2 日，为赵四（证件类型：身份证，证件号码：420105198503242420）在中国建设银行金融街支行办理的个人住
# 房商业贷款承担相关还款责任，责任人类型为保证人，相关还款责任金额 50,000（保证合同编号：100022233）。截至 2016
# 年 7 月 5 日，个人住房商业贷款余额 30,000。
# 2.2010 年 5 月 20 日，为李三（证件类型：身份证，证件号码：420105197503202424）在中国农业银行信用卡中心办理的贷记卡
# 承担相关还款责任，责任人类型为保证人，相关还款责任金额 50,000。截至 2016 年 7 月 20 日，贷记卡余额 20,000。
# 3.2012 年 3 月 2 日，为赵四（证件类型：身份证，证件号码：420105198503242420）在建元资本（中国）融资租赁有限公司办
# 理的约定购回式证券交易业务承担相关还款责任，责任人类型为其他，相关还款责任金额 50,000。截至 2016 年 7 月 5 日，
# 约定购回式证券交易余额 30,000。
# 4.2009 年 3 月 2 日，为北京某公司（证件类型：机构信用代码，证件号码：12011819008808）在建设银行北京分行办理的固定
# 资产贷款承担相关还款责任，责任人类型为应收账款债务人，相关还款责任金额 200,000。截至 2015 年 5 月 25 日，固定资
# 产贷款余额 30,000（人民币）。
# """

In [89]:
# cases = split_cases_by_idx_from_paragraph(raw)
# cases

['2009年3月2日，为赵四（证件类型：身份证，证件号码：420105198503242420）在中国建设银行金融街支行办理的个人住房商业贷款承担相关还款责任，责任人类型为保证人，相关还款责任金额50,000（保证合同编号：100022233）。截至2016年7月5日，个人住房商业贷款余额30,000。',
 '2010年5月20日，为李三（证件类型：身份证，证件号码：420105197503202424）在中国农业银行信用卡中心办理的贷记卡承担相关还款责任，责任人类型为保证人，相关还款责任金额50,000。截至2016年7月20日，贷记卡余额20,000。',
 '2012年3月2日，为赵四（证件类型：身份证，证件号码：420105198503242420）在建元资本（中国）融资租赁有限公司办理的约定购回式证券交易业务承担相关还款责任，责任人类型为其他，相关还款责任金额50,000。截至2016年7月5日，约定购回式证券交易余额30,000。',
 '2009年3月2日，为北京某公司（证件类型：机构信用代码，证件号码：12011819008808）在建设银行北京分行办理的固定资产贷款承担相关还款责任，责任人类型为应收账款债务人，相关还款责任金额200,000。截至2015年5月25日，固定资产贷款余额30,000（人民币）。']

In [92]:
xghkzrxx_data = ['2009年3月2日，为赵四（证件类型：身份证，证件号码：420105198503242420）在中国建设银行金融街支行办理的个人住房商业贷款承担相关还款责任，责任人类型为保证人，相关还款责任金额50,000（保证合同编号：100022233）。截至2016年7月5日，个人住房商业贷款余额30,000。',
 '2010年5月20日，为李三（证件类型：身份证，证件号码：420105197503202424）在中国农业银行信用卡中心办理的贷记卡承担相关还款责任，责任人类型为保证人，相关还款责任金额50,000。截至2016年7月20日，贷记卡余额20,000。',
 '2012年3月2日，为赵四（证件类型：身份证，证件号码：420105198503242420）在建元资本（中国）融资租赁有限公司办理的约定购回式证券交易业务承担相关还款责任，责任人类型为其他，相关还款责任金额50,000。截至2016年7月5日，约定购回式证券交易余额30,000。',
 '2009年3月2日，为北京某公司（证件类型：机构信用代码，证件号码：12011819008808）在建设银行北京分行办理的固定资产贷款承担相关还款责任，责任人类型为应收账款债务人，相关还款责任金额200,000。截至2015年5月25日，固定资产贷款余额30,000（人民币）。']

xghkzrxx_data

['2009年3月2日，为赵四（证件类型：身份证，证件号码：420105198503242420）在中国建设银行金融街支行办理的个人住房商业贷款承担相关还款责任，责任人类型为保证人，相关还款责任金额50,000（保证合同编号：100022233）。截至2016年7月5日，个人住房商业贷款余额30,000。',
 '2010年5月20日，为李三（证件类型：身份证，证件号码：420105197503202424）在中国农业银行信用卡中心办理的贷记卡承担相关还款责任，责任人类型为保证人，相关还款责任金额50,000。截至2016年7月20日，贷记卡余额20,000。',
 '2012年3月2日，为赵四（证件类型：身份证，证件号码：420105198503242420）在建元资本（中国）融资租赁有限公司办理的约定购回式证券交易业务承担相关还款责任，责任人类型为其他，相关还款责任金额50,000。截至2016年7月5日，约定购回式证券交易余额30,000。',
 '2009年3月2日，为北京某公司（证件类型：机构信用代码，证件号码：12011819008808）在建设银行北京分行办理的固定资产贷款承担相关还款责任，责任人类型为应收账款债务人，相关还款责任金额200,000。截至2015年5月25日，固定资产贷款余额30,000（人民币）。']

#### -- uie-base

In [110]:
xghkzrxx_results = []
for case in xghkzrxx_data:
    result = extract_info_from_taskflow(case=case, schema=xghkzrxx_schema)
    xghkzrxx_results.append(result)

In [111]:
i = 0
print(xghkzrxx_data[i])
display(pd.DataFrame(list(xghkzrxx_results[i].items()), columns=['key', 'value']))

2009年3月2日，为赵四（证件类型：身份证，证件号码：420105198503242420）在中国建设银行金融街支行办理的个人住房商业贷款承担相关还款责任，责任人类型为保证人，相关还款责任金额50,000（保证合同编号：100022233）。截至2016年7月5日，个人住房商业贷款余额30,000。


,key,value
0,业务办理日期,2009年3月2日
1,办理方姓名,赵四
2,办理人证件类型,身份证
3,办理人证件号码,420105198503242420
4,办理机构,中国建设银行金融街支行
5,承担相关还款责任的业务,None
6,责任人类型,保证人
7,责任金额,"50,000"
8,保证合同编号,100022233
9,截至日期,2016年7月5日


- ! 承担相关还款责任的业务

In [112]:
i = 1
print(xghkzrxx_data[i])
display(pd.DataFrame(list(xghkzrxx_results[i].items()), columns=['key', 'value']))

2010年5月20日，为李三（证件类型：身份证，证件号码：420105197503202424）在中国农业银行信用卡中心办理的贷记卡承担相关还款责任，责任人类型为保证人，相关还款责任金额50,000。截至2016年7月20日，贷记卡余额20,000。


,key,value
0,业务办理日期,2010年5月20日
1,办理方姓名,李三
2,办理人证件类型,身份证
3,办理人证件号码,420105197503202424
4,办理机构,中国农业银行信用卡中心
5,承担相关还款责任的业务,None
6,责任人类型,保证人
7,责任金额,"50,000"
8,保证合同编号,None
9,截至日期,2016年7月20日


- 承担相关还款责任的业务

In [113]:
i = 2
print(xghkzrxx_data[i])
display(pd.DataFrame(list(xghkzrxx_results[i].items()), columns=['key', 'value']))

2012年3月2日，为赵四（证件类型：身份证，证件号码：420105198503242420）在建元资本（中国）融资租赁有限公司办理的约定购回式证券交易业务承担相关还款责任，责任人类型为其他，相关还款责任金额50,000。截至2016年7月5日，约定购回式证券交易余额30,000。


,key,value
0,业务办理日期,2012年3月2日
1,办理方姓名,赵四
2,办理人证件类型,身份证
3,办理人证件号码,420105198503242420
4,办理机构,建元资本（中国）融资租赁有限公司
5,承担相关还款责任的业务,约定购回式证券交易业务
6,责任人类型,其他
7,责任金额,"50,000"
8,保证合同编号,None
9,截至日期,2016年7月5日


- ！余额数目	
- 承担相关还款责任的业务

In [114]:
i = 3
print(xghkzrxx_data[i])
display(pd.DataFrame(list(xghkzrxx_results[i].items()), columns=['key', 'value']))

2009年3月2日，为北京某公司（证件类型：机构信用代码，证件号码：12011819008808）在建设银行北京分行办理的固定资产贷款承担相关还款责任，责任人类型为应收账款债务人，相关还款责任金额200,000。截至2015年5月25日，固定资产贷款余额30,000（人民币）。


,key,value
0,业务办理日期,2009年3月2日
1,办理方姓名,北京某公司
2,办理人证件类型,机构信用代码
3,办理人证件号码,12011819008808
4,办理机构,建设银行北京分行
5,承担相关还款责任的业务,None
6,责任人类型,应收账款债务人
7,责任金额,"200,000"
8,保证合同编号,None
9,截至日期,2015年5月25日


- 余额数目
- ！余额币种

#### -- fix

In [130]:
xghkzrxx_results_fixed = []
for case, result in zip(xghkzrxx_data, xghkzrxx_results):
    result = text_result_fixing(case, result)
    xghkzrxx_results_fixed.append(result)

In [131]:
for case, result in zip(xghkzrxx_data, xghkzrxx_results_fixed):
    print(case)
    display(pd.DataFrame(list(result.items()), columns=['key', 'value']))

2009年3月2日，为赵四（证件类型：身份证，证件号码：420105198503242420）在中国建设银行金融街支行办理的个人住房商业贷款承担相关还款责任，责任人类型为保证人，相关还款责任金额50,000（保证合同编号：100022233）。截至2016年7月5日，个人住房商业贷款余额30,000。


,key,value
0,业务办理日期,2009年3月2日
1,办理方姓名,赵四
2,办理人证件类型,身份证
3,办理人证件号码,420105198503242420
4,办理机构,中国建设银行金融街支行
5,承担相关还款责任的业务,个人住房商业贷款
6,责任人类型,保证人
7,责任金额,"50,000"
8,保证合同编号,100022233
9,截至日期,2016年7月5日


2010年5月20日，为李三（证件类型：身份证，证件号码：420105197503202424）在中国农业银行信用卡中心办理的贷记卡承担相关还款责任，责任人类型为保证人，相关还款责任金额50,000。截至2016年7月20日，贷记卡余额20,000。


,key,value
0,业务办理日期,2010年5月20日
1,办理方姓名,李三
2,办理人证件类型,身份证
3,办理人证件号码,420105197503202424
4,办理机构,中国农业银行信用卡中心
5,承担相关还款责任的业务,贷记卡
6,责任人类型,保证人
7,责任金额,"50,000"
8,保证合同编号,None
9,截至日期,2016年7月20日


2012年3月2日，为赵四（证件类型：身份证，证件号码：420105198503242420）在建元资本（中国）融资租赁有限公司办理的约定购回式证券交易业务承担相关还款责任，责任人类型为其他，相关还款责任金额50,000。截至2016年7月5日，约定购回式证券交易余额30,000。


,key,value
0,业务办理日期,2012年3月2日
1,办理方姓名,赵四
2,办理人证件类型,身份证
3,办理人证件号码,420105198503242420
4,办理机构,建元资本（中国）融资租赁有限公司
5,承担相关还款责任的业务,约定购回式证券交易业务
6,责任人类型,其他
7,责任金额,"50,000"
8,保证合同编号,None
9,截至日期,2016年7月5日


2009年3月2日，为北京某公司（证件类型：机构信用代码，证件号码：12011819008808）在建设银行北京分行办理的固定资产贷款承担相关还款责任，责任人类型为应收账款债务人，相关还款责任金额200,000。截至2015年5月25日，固定资产贷款余额30,000（人民币）。


,key,value
0,业务办理日期,2009年3月2日
1,办理方姓名,北京某公司
2,办理人证件类型,机构信用代码
3,办理人证件号码,12011819008808
4,办理机构,建设银行北京分行
5,承担相关还款责任的业务,固定资产贷款
6,责任人类型,应收账款债务人
7,责任金额,"200,000"
8,保证合同编号,None
9,截至日期,2015年5月25日


#### -- en

In [138]:
xghkzrxx_results_en = []

for result in xghkzrxx_results_fixed:
    result_en = {key_2_English(k):v for k,v in result.items()}
    xghkzrxx_results_en.append(result_en)

xghkzrxx_results_en

[{'business_handling_date': '2009年3月2日',
  'handler_name': '赵四',
  'handler_id_type': '身份证',
  'handler_id_number': '420105198503242420',
  'handling_institution': '中国建设银行金融街支行',
  'business_bearing_relevant_repayment_responsibilities': '个人住房商业贷款',
  'responsible_person_type': '保证人',
  'amount_liable': '50,000',
  'guarantee_contract_no': '100022233',
  'reference_date': '2016年7月5日',
  'remaining_balance': '30,000',
  'balance_currency': None},
 {'business_handling_date': '2010年5月20日',
  'handler_name': '李三',
  'handler_id_type': '身份证',
  'handler_id_number': '420105197503202424',
  'handling_institution': '中国农业银行信用卡中心',
  'business_bearing_relevant_repayment_responsibilities': '贷记卡',
  'responsible_person_type': '保证人',
  'amount_liable': '50,000',
  'guarantee_contract_no': None,
  'reference_date': '2016年7月20日',
  'remaining_balance': '20,000',
  'balance_currency': None},
 {'business_handling_date': '2012年3月2日',
  'handler_name': '赵四',
  'handler_id_type': '身份证',
  'handler_id_numbe

# 非信贷交易记录

In [77]:
## 清一下内存
import gc
gc.collect()

3020

```json
{
        'header-2':{
        'header-name':'非信贷交易记录',
        'content':'这部分包含您最近 5 年内的非信贷交易记录。金额类数据均以人民币计算，精确到元。'
        'section':{
            'header-2-0':{...},
            'header-2-1':{...},
            ...
        }
    },
}
```

## 后付费记录

### == 内容样例

![后付费记录](4_简版方案/后付费记录.png)

In [75]:
# ## 预期ocr结果
# hffjl_data = []
# hffjl_data.append(['机构名称：中国电信北京分公司', '业务类型：固定电话后付费', '记账年月：2016年7月'])
# hffjl_data.append(['业务开通日期：2012年6月28日', '当前缴费状态：欠费', '当前欠费金额：550'])
# hffjl_data.append(['机构名称：中国移动北京分公司', '业务类型：移动电话后付费', '记账年月：2016年6月'])
# hffjl_data.append(['业务开通日期：2013年1月31日', '当前缴费状态：正常', '当前欠费金额：0'])
# hffjl_data.append(['机构名称：北京自来水公司', '业务类型：自来水费', '记账年月：2016年6月'])
# hffjl_data.append(['业务开通日期：2014年1月31日', '当前缴费状态：正常', '当前欠费金额：0'])
# hffjl_data

[['机构名称：中国电信北京分公司', '业务类型：固定电话后付费', '记账年月：2016年7月'],
 ['业务开通日期：2012年6月28日', '当前缴费状态：欠费', '当前欠费金额：550'],
 ['机构名称：中国移动北京分公司', '业务类型：移动电话后付费', '记账年月：2016年6月'],
 ['业务开通日期：2013年1月31日', '当前缴费状态：正常', '当前欠费金额：0'],
 ['机构名称：北京自来水公司', '业务类型：自来水费', '记账年月：2016年6月'],
 ['业务开通日期：2014年1月31日', '当前缴费状态：正常', '当前欠费金额：0']]

In [76]:
# """ 预处按条目理分割data """
# def hffjl_split_cases(data):  ## 后付费记录
#     cases = []
#     n_line_split = 2  ## 固定2行一组
#     for i in range(len(data)):
#         if i % n_line_split == 0:
#             cases.append(data[i])
#         else:
#             cases[-1].extend(data[i])
#     return cases

# hffjl_cases = hffjl_split_cases(data=hffjl_data)
# hffjl_cases

[['机构名称：中国电信北京分公司',
  '业务类型：固定电话后付费',
  '记账年月：2016年7月',
  '业务开通日期：2012年6月28日',
  '当前缴费状态：欠费',
  '当前欠费金额：550'],
 ['机构名称：中国移动北京分公司',
  '业务类型：移动电话后付费',
  '记账年月：2016年6月',
  '业务开通日期：2013年1月31日',
  '当前缴费状态：正常',
  '当前欠费金额：0'],
 ['机构名称：北京自来水公司',
  '业务类型：自来水费',
  '记账年月：2016年6月',
  '业务开通日期：2014年1月31日',
  '当前缴费状态：正常',
  '当前欠费金额：0']]

!!! 更新

In [98]:
## 预期ocr结果
hffjl_data = []
hffjl_data.append(['机构名称：中国电信北京分公司', '业务类型：固定电话后付费', '记账年月：2016年7月', '业务开通日期：2012年6月28日', '当前缴费状态：欠费', '当前欠费金额：550'])
hffjl_data.append(['机构名称：中国移动北京分公司', '业务类型：移动电话后付费', '记账年月：2016年6月', '业务开通日期：2013年1月31日', '当前缴费状态：正常', '当前欠费金额：0'])
hffjl_data.append(['机构名称：北京自来水公司', '业务类型：自来水费', '记账年月：2016年6月', '业务开通日期：2014年1月31日', '当前缴费状态：正常', '当前欠费金额：0'])
hffjl_data

[['机构名称：中国电信北京分公司',
  '业务类型：固定电话后付费',
  '记账年月：2016年7月',
  '业务开通日期：2012年6月28日',
  '当前缴费状态：欠费',
  '当前欠费金额：550'],
 ['机构名称：中国移动北京分公司',
  '业务类型：移动电话后付费',
  '记账年月：2016年6月',
  '业务开通日期：2013年1月31日',
  '当前缴费状态：正常',
  '当前欠费金额：0'],
 ['机构名称：北京自来水公司',
  '业务类型：自来水费',
  '记账年月：2016年6月',
  '业务开通日期：2014年1月31日',
  '当前缴费状态：正常',
  '当前欠费金额：0']]

### == 预期提取效果

整体（提取结果存放在字段 `info_extraction`）：
```json
{
    'header-2-0':{
        'header-name':'后付费记录',
        'content':[ocr返回结果],
        'info_extraction':[case_0_info, case_1_info, case_2_info],
    }
}
```
`case_i_info` 关键字段及其翻译如下：

In [78]:
'''
此处schema应由规则提取得到，
而非人为规定
'''
hffjl_schema = ["机构名称", "业务类型", "记账年月"]
hffjl_schema.extend(["业务开通日期", "当前缴费状态", "当前欠费金额"])

case_i_info = {k:key_2_English(k) for k in hffjl_schema}
case_i_info

{'机构名称': 'organization_name',
 '业务类型': 'business_type',
 '记账年月': 'accounting_year_month',
 '业务开通日期': 'business_activation_date',
 '当前缴费状态': 'current_billing_status',
 '当前欠费金额': 'current_arrears'}

### == 提取方案

- 手动规则提取
    - 先识别出一个一个案例（e.g. 固定两行为一组）
    - 键值对抽取：" "/": "/":"进行分割, 前面为key, 后面为value

### == 案例执行结果

In [100]:
def hffjl_extract_info_from_caselist(cases):
    results = []
    for case in cases:
        result = {}
        for k_v in case:
            if k_v =='':
                continue
            k_v = re.sub(r'[:：]', '|', k_v)  ## 冒号分割
            k_v = re.split(r'\|+', k_v)
            k = re.sub(r'\s+', '', k_v[0])  ## 去除空白
            v = re.sub(r'\s+', '', k_v[1])
            result[k] = v
        results.append(result)
    return results

hffjl_results = hffjl_extract_info_from_caselist(hffjl_data)

In [102]:
hffjl_results_en = []
for i in range (len(hffjl_data)):
    print("--------------")
    print(hffjl_data[i])
    result = {key_2_English(k):v for k,v in hffjl_results[i].items()}
    hffjl_results_en.append(result)
    display(pd.DataFrame(list(result.items()), columns=['key', 'value']))

--------------
['机构名称：中国电信北京分公司', '业务类型：固定电话后付费', '记账年月：2016年7月', '业务开通日期：2012年6月28日', '当前缴费状态：欠费', '当前欠费金额：550']


,key,value
0,organization_name,中国电信北京分公司
1,business_type,固定电话后付费
2,accounting_year_month,2016年7月
3,business_activation_date,2012年6月28日
4,current_billing_status,欠费
5,current_arrears,550


--------------
['机构名称：中国移动北京分公司', '业务类型：移动电话后付费', '记账年月：2016年6月', '业务开通日期：2013年1月31日', '当前缴费状态：正常', '当前欠费金额：0']


,key,value
0,organization_name,中国移动北京分公司
1,business_type,移动电话后付费
2,accounting_year_month,2016年6月
3,business_activation_date,2013年1月31日
4,current_billing_status,正常
5,current_arrears,0


--------------
['机构名称：北京自来水公司', '业务类型：自来水费', '记账年月：2016年6月', '业务开通日期：2014年1月31日', '当前缴费状态：正常', '当前欠费金额：0']


,key,value
0,organization_name,北京自来水公司
1,business_type,自来水费
2,accounting_year_month,2016年6月
3,business_activation_date,2014年1月31日
4,current_billing_status,正常
5,current_arrears,0


In [103]:
hffjl_results_en

[{'organization_name': '中国电信北京分公司',
  'business_type': '固定电话后付费',
  'accounting_year_month': '2016年7月',
  'business_activation_date': '2012年6月28日',
  'current_billing_status': '欠费',
  'current_arrears': '550'},
 {'organization_name': '中国移动北京分公司',
  'business_type': '移动电话后付费',
  'accounting_year_month': '2016年6月',
  'business_activation_date': '2013年1月31日',
  'current_billing_status': '正常',
  'current_arrears': '0'},
 {'organization_name': '北京自来水公司',
  'business_type': '自来水费',
  'accounting_year_month': '2016年6月',
  'business_activation_date': '2014年1月31日',
  'current_billing_status': '正常',
  'current_arrears': '0'}]

### == 备注

- 具体操作需根据ocr返回结果格式调整

# 公共记录

## 欠税记录
略 -- 方案同 `后付费记录`

- 固定两行一组
- 预期识别关键字段名称:

In [59]:
qsjl_schema = ['主管税务机关', '欠税统计日期', '欠税总额', '纳税人识别号']
{k:key_2_English(k) for k in qsjl_schema}

{'主管税务机关': 'tax_authorities',
 '欠税统计日期': 'tax_arrears_statistics_date',
 '欠税总额': 'total_tax_arrears',
 '纳税人识别号': 'taxpayer_id'}

## 民事判决记录
略 -- 方案同 `后付费记录`

- 固定5行一组
- 预期识别关键字段名称：

In [63]:
mspjjl_schema = ['立案法院', '案号', '案由', '结案方式', '立案日期', '判决/调解结果', '诉讼标的', '判决/调解生效日期', '诉讼标的金额']
{k:key_2_English(k) for k in mspjjl_schema}

{'立案法院': 'filing_court',
 '案号': 'case_number',
 '案由': 'case_reason',
 '结案方式': 'case_closure_method',
 '立案日期': 'filing_date',
 '判决/调解结果': 'judgment_or_mediation_result',
 '诉讼标的': 'litigation_subject',
 '判决/调解生效日期': 'judgment_or_mediation_effective_date',
 '诉讼标的金额': 'litigation_amount'}

## 强制执行记录
略 -- 方案同 `后付费记录`

- 固定6行一组
- 预期识别关键字段名称：

In [66]:
qzzxjl_schema = ['执行法院', '案号', '执行案由', '结案方式', '立案日期', '案件状态', '申请执行标的', '已执行标的', '申请执行标的金额', '已执行标的金额', '结案日期']

{k:key_2_English(k) for k in qzzxjl_schema}

{'执行法院': 'executing_court',
 '案号': 'case_number',
 '执行案由': 'reason_for_execution',
 '结案方式': 'case_closure_method',
 '立案日期': 'filing_date',
 '案件状态': 'case_status',
 '申请执行标的': 'claimed_execution_subject',
 '已执行标的': 'executed_subject',
 '申请执行标的金额': 'claimed_execution_amount',
 '已执行标的金额': 'executed_amount',
 '结案日期': 'closure_date'}

## 行政处罚记录
略 -- 方案同 `后付费记录`

- 固定4行一组
- 预期识别关键字段名称：

In [70]:
xzcfjl_schema = ['处罚机构', '文书编号', '处罚内容', '行政复议结果', '处罚金额', '生效日期', '截止日期']

{k:key_2_English(k) for k in xzcfjl_schema}

{'处罚机构': 'penalty_authority',
 '文书编号': 'document_number',
 '处罚内容': 'penalty_details',
 '行政复议结果': 'administrative_review_result',
 '处罚金额': 'penalty_amount',
 '生效日期': 'effective_date',
 '截止日期': 'due_date'}

# 查询记录

In [93]:
def cxjl_extract_info_from_table(table:list[list], need_translate=True):
    '''
    适用于 第一行为列名, 无行名 的表格
    '''
    col_names = table[0]
    if need_translate:
        col_names = key_list_2_English(col_names)

    results = []
    for r in range(1, len(table)):
        result = {}
        for c in range(len(col_names)):
            result[col_names[c]] = table[r][c]
        results.append(result)

    return results

## 机构查询记录明细 

### == 内容样例

![机构查询记录](4_简版方案/机构查询记录.png)

In [96]:
jgcsjl_data = [
    ['编号', '查询日期', '查询机构', '查询原因'],
    ['1', '2016年7月15日', '中国工商银行', '贷后管理'],
    ['2', '2015年12月10日', '中国农业银行', '贷款审批'],
    ['3', '2015年2月12日', '中国农业银行', '担保资格审查'],
    ['4', '2015年2月10日', '建元资本', '融资审批'],
    ['5', '2015年2月1日', '平安财险', '保前审查']
]

jgcsjl_data

[['编号', '查询日期', '查询机构', '查询原因'],
 ['1', '2016年7月15日', '中国工商银行', '贷后管理'],
 ['2', '2015年12月10日', '中国农业银行', '贷款审批'],
 ['3', '2015年2月12日', '中国农业银行', '担保资格审查'],
 ['4', '2015年2月10日', '建元资本', '融资审批'],
 ['5', '2015年2月1日', '平安财险', '保前审查']]

### == 预期提取效果

整体（提取结果存放在字段 `info_extraction`）：
```json
{
    'header-4-0':{
        'header-name':'机构查询记录明细',
        'content':[ocr返回结果],
        'info_extraction':[case_0_info, case_1_info, ..., case_4_info],
    }
}
```
`case_i_info` 关键字段及其翻译如下：

In [82]:
'''
此处schema应由规则提取得到，
而非人为规定
'''
csjl_schema = ['编号', '查询日期', '查询机构', '查询原因']

case_i_info = {k:key_2_English(k) for k in csjl_schema}
case_i_info

{'编号': 'item_no',
 '查询日期': 'inquiry_date',
 '查询机构': 'inquiry_entity',
 '查询原因': 'inquiry_reason'}

### == 提取方案

- 默认第一行为列名
- 默认第二行开始每一行为一份数据

### == 案例执行效果

In [97]:
jgcsjl_info_extraction = cxjl_extract_info_from_table(jgcsjl_data)
jgcsjl_info_extraction

[{'item_no': '1',
  'inquiry_date': '2016年7月15日',
  'inquiry_entity': '中国工商银行',
  'inquiry_reason': '贷后管理'},
 {'item_no': '2',
  'inquiry_date': '2015年12月10日',
  'inquiry_entity': '中国农业银行',
  'inquiry_reason': '贷款审批'},
 {'item_no': '3',
  'inquiry_date': '2015年2月12日',
  'inquiry_entity': '中国农业银行',
  'inquiry_reason': '担保资格审查'},
 {'item_no': '4',
  'inquiry_date': '2015年2月10日',
  'inquiry_entity': '建元资本',
  'inquiry_reason': '融资审批'},
 {'item_no': '5',
  'inquiry_date': '2015年2月1日',
  'inquiry_entity': '平安财险',
  'inquiry_reason': '保前审查'}]

### == 备注

- 具体操作需根据ocr返回结果格式调整

## 本人查询记录明细
略 -- 方案同`机构查新记录明细`

- 默认第一行为列名，无行名
- 预期提取字段：

In [101]:
csjl_schema = ['编号', '查询日期', '查询机构', '查询原因']

{k:key_2_English(k) for k in csjl_schema}

{'编号': 'item_no',
 '查询日期': 'inquiry_date',
 '查询机构': 'inquiry_entity',
 '查询原因': 'inquiry_reason'}